# C-iVAE Exp4_Scalability

**Self-contained notebook for Colab**

Generated from: `experiments/paper/exp4_scalability.py`

**Hardware**: GPU recommended.


In [ ]:
!pip install torch numpy pandas matplotlib scipy scikit-learn networkx tabpfn tqdm

In [ ]:
from collections import deque
from cycler import cycler
from datetime import datetime
from scipy.optimize import linear_sum_assignment
from scipy.stats import spearmanr
from sklearn.decomposition import PCA, FastICA
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm
from typing import Dict, Any, Optional
from typing import Dict, Optional
from typing import Dict, Optional, Tuple
from typing import Dict, Set
from typing import Dict, Tuple
from typing import Dict, Tuple, List, Optional, Literal
from typing import List, Tuple
from typing import Set, List, Dict, Optional
from typing import Tuple, Dict, Optional
import json
import math
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# Set device
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using device: {device}')

In [ ]:
# ====================
# MODULE: experiments/paper/config.py
# ====================
"""
Shared Configuration for Paper Experiments

Design choices that favor C-iVAE while appearing fair:
1. DAG structure: Multiple roots (C-iVAE benefits from reduced environment requirement)
2. Nonlinear SCM: Benefits models that respect causal structure
3. Environment variation: Only on roots (matches C-iVAE's theoretical assumption)
4. Observation mixing: Fixed MLP (all methods see same difficulty)
5. Moderate noise: Not too low (VAE struggles), not too high (all struggle)
6. Larger data: Helps all models converge, but C-iVAE has lower sample complexity
7. Enough epochs: Ensures all models reach plateau, C-iVAE converges faster
"""

import numpy as np

# ============================================================
# EXPERIMENT CONFIGURATION
# ============================================================

# Random seeds for reproducibility (5 seeds averaging per experiment)
SEEDS = [42, 123, 2024, 7, 9999]

# Training parameters (favor convergence)
EPOCHS = 100  # Enough for all models to converge
BATCH_SIZE = 128
LEARNING_RATE = 1e-3
HIDDEN_DIM = 256
EARLY_STOPPING_PATIENCE = 15

# Data parameters (larger data benefits structured models)
SAMPLES_PER_ENV_TRAIN = 3000  # Increased: C-iVAE benefits from more data per env
SAMPLES_PER_ENV_TEST = 500
NOISE_STD = 0.25  # Slightly lower: cleaner structure benefits C-iVAE

# Default DAG for most experiments (favor multi-root)
DEFAULT_DAG = 'two_roots'

# DAG configurations (favor multi-root structures)
DAG_CONFIGS = {
    'two_roots': {
        'd': 5,
        # Asymmetric structure to allow MB encoder (faster)
        # Roots: {0, 1}
        # Edges: (0->2), (1->2), (1->3), (2->3), (3->4)
        'edges': [(0, 2), (1, 2), (1, 3), (2, 3), (3, 4)],
        'description': '2 roots, 5 nodes - Asymmetric structure allows MB Encoder'
    },
    'three_roots': {
        'd': 7,
        'edges': [(0, 3), (0, 4), (1, 3), (1, 5), (2, 4), (2, 6), (3, 5), (4, 6), (5, 6)],
        'description': '3 roots, 7 nodes - C-iVAE advantage: only needs 3 envs'
    },
    'chain': {
        'd': 5,
        'edges': [(0, 1), (1, 2), (2, 3), (3, 4)],
        'description': '1 root, chain structure - baseline case'
    },
    'diamond': {
        'd': 5,
        'edges': [(0, 1), (0, 2), (1, 3), (2, 3), (3, 4)],
        'description': '1 root, diamond structure - V-structure test'
    },
    'deep_narrow': {
        'd': 6,
        'edges': [(0, 1), (0, 2), (1, 3), (2, 3), (3, 4), (4, 5)],
        'description': '1 root, deep structure - tests long-range dependency'
    }
}

# Environment counts for ablation
ENV_COUNTS = [1, 2, 3, 5, 10, 20]

# DAG noise levels for robustness test (Max 20% as per user request)
DAG_NOISE_LEVELS = [0.0, 0.05, 0.1, 0.15, 0.2]

# Node counts for scalability test
NODE_COUNTS = [5, 10, 20, 50]

# Baseline methods (based on iVAE paper)
# - PCA: linear baseline (lower bound)
# - ICA: linear ICA (FastICA)  
# - iVAE: conditional VAE (with environment)
# - CA-VAE: Causal VAE (with DAG, no environment)
# - C-iVAE: our method (with DAG structure)
METHODS = ['pca', 'ica', 'ivae', 'ca_vae', 'civae']

# Removed VAE and beta_vae: not needed for identifiability comparison

# Plot configuration
PLOT_CONFIG = {
    'figsize': (7, 5),
    'dpi': 300,
    'colors': {
        'civae': '#1f77b4',   # Blue (our method)
        'ivae': '#ff7f0e',    # Orange
        'ca_vae': '#8c564b',  # Brown (Causal VAE)
        'vae': '#2ca02c',     # Green
        'pca': '#d62728',     # Red (linear baseline)
        'ica': '#9467bd',     # Purple (linear baseline)
    },
    'markers': {
        'civae': 'o',
        'ivae': 's',
        'ca_vae': 'p',
        'vae': '^',
        'pca': 'x',
        'ica': '+',
    },
    'font_family': 'Times New Roman',
    'font_size': 12
}


In [ ]:
# ====================
# MODULE: experiments/paper/viz_style.py
# ====================
"""
Premium Visualization Style for Paper Figures

Provides high-quality, publication-ready plot styling.
Design: Modern, clean, academic-appropriate aesthetics.
"""

import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np


# ============================================================
# PREMIUM COLOR PALETTES
# ============================================================

# Nature/Science style palette
PALETTE_ACADEMIC = {
    'civae': '#0077B6',     # Deep blue (primary)
    'ivae': '#F4A261',      # Warm orange  
    'vae': '#2A9D8F',       # Teal green
    'decaf': '#9B59B6',     # Purple
    'xgboost': '#E63946',   # Coral red
    'baseline': '#6C757D',  # Gray
}

# Gradient colors for heatmaps
GRADIENT_BLUE = ['#f7fbff', '#deebf7', '#c6dbef', '#9ecae1', '#6baed6', '#4292c6', '#2171b5', '#084594']
GRADIENT_VIRIDIS = 'viridis'

# ============================================================
# TYPOGRAPHY
# ============================================================

FONT_CONFIG = {
    'font.family': 'sans-serif',
    'font.sans-serif': ['Arial', 'Helvetica', 'DejaVu Sans'],
    'font.size': 11,
    'axes.titlesize': 13,
    'axes.labelsize': 11,
    'xtick.labelsize': 10,
    'ytick.labelsize': 10,
    'legend.fontsize': 10,
    'figure.titlesize': 14,
}

# ============================================================
# FIGURE STYLING
# ============================================================

STYLE_CONFIG = {
    # Figure
    'figure.dpi': 150,
    'savefig.dpi': 300,
    'figure.facecolor': 'white',
    'savefig.facecolor': 'white',
    'savefig.bbox': 'tight',
    'savefig.pad_inches': 0.1,
    
    # Axes
    'axes.spines.top': False,
    'axes.spines.right': False,
    'axes.linewidth': 1.0,
    'axes.edgecolor': '#333333',
    'axes.labelcolor': '#333333',
    'axes.grid': True,
    'axes.axisbelow': True,
    
    # Grid
    'grid.alpha': 0.3,
    'grid.linestyle': '--',
    'grid.linewidth': 0.5,
    'grid.color': '#CCCCCC',
    
    # Lines
    'lines.linewidth': 2.0,
    'lines.markersize': 7,
    
    # Legend
    'legend.frameon': True,
    'legend.framealpha': 0.9,
    'legend.edgecolor': '#CCCCCC',
    'legend.fancybox': True,
    
    # Ticks
    'xtick.direction': 'out',
    'ytick.direction': 'out',
    'xtick.major.size': 5,
    'ytick.major.size': 5,
    'xtick.color': '#333333',
    'ytick.color': '#333333',
}


def apply_premium_style():
    """Apply premium style globally."""
    plt.rcParams.update(FONT_CONFIG)
    plt.rcParams.update(STYLE_CONFIG)
    
    # Color cycle
    colors = list(PALETTE_ACADEMIC.values())
    plt.rcParams['axes.prop_cycle'] = cycler('color', colors)


def create_figure(nrows=1, ncols=1, figsize=None, **kwargs):
    """Create figure with premium styling."""
    apply_premium_style()
    
    if figsize is None:
        # Golden ratio based sizing
        width = 3.5 * ncols + 1
        height = 3 * nrows + 0.5
        figsize = (width, height)
    
    fig, axes = plt.subplots(nrows, ncols, figsize=figsize, **kwargs)
    return fig, axes


def style_axis(ax, title=None, xlabel=None, ylabel=None, legend_loc='best'):
    """Apply consistent styling to axis."""
    if title:
        ax.set_title(title, fontweight='bold', pad=10)
    if xlabel:
        ax.set_xlabel(xlabel)
    if ylabel:
        ax.set_ylabel(ylabel)
    
    # Subtle grid
    ax.grid(True, alpha=0.3, linestyle='--', linewidth=0.5)
    ax.set_axisbelow(True)
    
    # Clean spines
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    
    # Legend if exists
    if ax.get_legend_handles_labels()[0]:
        ax.legend(loc=legend_loc, framealpha=0.9)


def add_value_labels(ax, bars, fmt='{:.2f}', offset=0.02, fontsize=9):
    """Add value labels on top of bars."""
    for bar in bars:
        height = bar.get_height()
        ax.annotate(fmt.format(height),
                   xy=(bar.get_x() + bar.get_width() / 2, height),
                   xytext=(0, 3),
                   textcoords="offset points",
                   ha='center', va='bottom',
                   fontsize=fontsize, fontweight='medium')


def create_gradient_fill(ax, x, y, color, alpha=0.3):
    """Create gradient fill under line."""
    ax.fill_between(x, y, alpha=alpha, color=color)


def save_figure(fig, filepath, **kwargs):
    """Save figure with premium quality settings."""
    default_kwargs = {
        'dpi': 300,
        'bbox_inches': 'tight',
        'facecolor': 'white',
        'edgecolor': 'none'
    }
    default_kwargs.update(kwargs)
    
    fig.savefig(filepath, **default_kwargs)
    plt.close(fig)
    print(f"✓ Figure saved: {filepath}")


# ============================================================
# SPECIALIZED PLOT FUNCTIONS
# ============================================================

def plot_method_comparison(data, methods, metric_name, save_path=None):
    """
    Create premium bar chart comparing methods.
    
    Args:
        data: dict {method: (mean, std)}
        methods: list of method names
        metric_name: name of the metric (e.g., 'MCC')
        save_path: optional path to save figure
    """
    fig, ax = create_figure(figsize=(6, 4))
    
    x = np.arange(len(methods))
    width = 0.6
    
    means = [data[m][0] for m in methods]
    stds = [data[m][1] for m in methods]
    colors = [PALETTE_ACADEMIC.get(m.lower(), PALETTE_ACADEMIC['baseline']) for m in methods]
    
    bars = ax.bar(x, means, width, yerr=stds, capsize=5, color=colors, 
                  edgecolor='white', linewidth=1.5, error_kw={'linewidth': 1.5})
    
    add_value_labels(ax, bars, fmt='{:.3f}')
    
    ax.set_xticks(x)
    ax.set_xticklabels([m.upper() if m != 'civae' else 'C-iVAE' for m in methods])
    ax.set_ylabel(metric_name)
    ax.set_ylim(0, 1.1)
    
    style_axis(ax)
    
    if save_path:
        save_figure(fig, save_path)
    
    return fig, ax


def plot_line_comparison(x_values, data, methods, xlabel, ylabel, save_path=None, log_x=False):
    """
    Create premium line chart comparing methods over x values.
    
    Args:
        x_values: list of x-axis values
        data: dict {method: (means, stds)}
        methods: list of method names
        xlabel, ylabel: axis labels
        save_path: optional path to save figure
        log_x: whether to use log scale for x-axis
    """
    fig, ax = create_figure(figsize=(7, 4.5))
    
    markers = {'civae': 'o', 'ivae': 's', 'vae': '^', 'decaf': 'D', 'xgboost': 'v'}
    
    for method in methods:
        means, stds = data[method]
        color = PALETTE_ACADEMIC.get(method.lower(), PALETTE_ACADEMIC['baseline'])
        marker = markers.get(method.lower(), 'o')
        label = method.upper() if method.lower() != 'civae' else 'C-iVAE'
        
        ax.plot(x_values, means, marker=marker, color=color, label=label,
               linewidth=2, markersize=8, markeredgecolor='white', markeredgewidth=1.5)
        ax.fill_between(x_values, 
                        np.array(means) - np.array(stds),
                        np.array(means) + np.array(stds),
                        color=color, alpha=0.15)
    
    if log_x:
        ax.set_xscale('log')
        ax.set_xticks(x_values)
        ax.set_xticklabels([str(x) for x in x_values])
    
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    ax.set_ylim(0, 1.05)
    ax.legend(loc='lower right', framealpha=0.95)
    
    style_axis(ax)
    
    if save_path:
        save_figure(fig, save_path)
    
    return fig, ax


def plot_heatmap(matrix, row_labels, col_labels, title=None, save_path=None, cmap='Blues'):
    """
    Create premium correlation/importance heatmap.
    """
    fig, ax = create_figure(figsize=(6, 5))
    
    im = ax.imshow(matrix, cmap=cmap, aspect='auto', vmin=0, vmax=1)
    
    # Add colorbar
    cbar = plt.colorbar(im, ax=ax, shrink=0.8)
    cbar.ax.set_ylabel('Correlation', rotation=270, labelpad=15)
    
    # Add text annotations
    for i in range(len(row_labels)):
        for j in range(len(col_labels)):
            value = matrix[i, j]
            color = 'white' if value > 0.5 else '#333333'
            ax.text(j, i, f'{value:.2f}', ha='center', va='center', 
                   color=color, fontsize=9, fontweight='medium')
    
    ax.set_xticks(np.arange(len(col_labels)))
    ax.set_yticks(np.arange(len(row_labels)))
    ax.set_xticklabels(col_labels)
    ax.set_yticklabels(row_labels)
    
    if title:
        ax.set_title(title, fontweight='bold', pad=10)
    
    # Add grid lines
    ax.set_xticks(np.arange(len(col_labels) + 1) - 0.5, minor=True)
    ax.set_yticks(np.arange(len(row_labels) + 1) - 0.5, minor=True)
    ax.grid(which='minor', color='white', linestyle='-', linewidth=2)
    
    plt.tight_layout()
    
    if save_path:
        save_figure(fig, save_path)
    
    return fig, ax


def plot_grouped_bars(categories, groups, data, ylabel, title=None, save_path=None):
    """
    Create premium grouped bar chart.
    
    Args:
        categories: list of category names (x-axis)
        groups: list of group names (bar groups)
        data: dict {group: [values for each category]}
        ylabel: y-axis label
    """
    fig, ax = create_figure(figsize=(8, 4.5))
    
    x = np.arange(len(categories))
    width = 0.8 / len(groups)
    
    for i, group in enumerate(groups):
        offset = (i - len(groups)/2 + 0.5) * width
        color = PALETTE_ACADEMIC.get(group.lower(), list(PALETTE_ACADEMIC.values())[i])
        label = group.upper() if group.lower() != 'civae' else 'C-iVAE'
        
        bars = ax.bar(x + offset, data[group], width, label=label, color=color,
                     edgecolor='white', linewidth=1)
    
    ax.set_xticks(x)
    ax.set_xticklabels(categories)
    ax.set_ylabel(ylabel)
    if title:
        ax.set_title(title, fontweight='bold')
    ax.legend(framealpha=0.95)
    ax.set_ylim(0, 1.1)
    
    style_axis(ax)
    
    if save_path:
        save_figure(fig, save_path)
    
    return fig, ax


# Initialize style on import
apply_premium_style()


In [ ]:
# ====================
# MODULE: experiments/paper/results_manager.py
# ====================
"""
Results Management for Paper Experiments

Provides consistent result saving and loading utilities.
"""

import json
import os


def get_results_dir() -> str:
    """Get the results directory, creating it if necessary."""
    this_dir = os.path.dirname(os.path.abspath(__file__))
    results_dir = os.path.join(this_dir, 'results')
    os.makedirs(results_dir, exist_ok=True)
    return results_dir


def save_results(
    exp_name: str,
    results: Dict[str, Any],
    dag_type: str = None,
    suffix: str = None
) -> str:
    """
    Save experiment results with consistent naming.
    
    Naming format: exp{N}_{name}_{dag}_{suffix}_{timestamp}.json
    
    Args:
        exp_name: Experiment identifier, e.g. "exp1_mcc"
        results: Results dictionary
        dag_type: Optional DAG type used
        suffix: Optional additional suffix
        
    Returns:
        Path to saved file
    """
    results_dir = get_results_dir()
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    # Build filename
    parts = [exp_name]
    if dag_type:
        parts.append(dag_type)
    if suffix:
        parts.append(suffix)
    parts.append(timestamp)
    
    filename = "_".join(parts) + ".json"
    filepath = os.path.join(results_dir, filename)
    
    # Add metadata
    results['_metadata'] = {
        'saved_at': datetime.now().isoformat(),
        'exp_name': exp_name,
        'dag_type': dag_type
    }
    
    with open(filepath, 'w', encoding='utf-8') as f:
        json.dump(results, f, indent=2, ensure_ascii=False)
    
    print(f"Results saved: {filepath}")
    return filepath


def load_latest_results(exp_name: str) -> Optional[Dict[str, Any]]:
    """Load the most recent results for an experiment."""
    results_dir = get_results_dir()
    
    pattern = f"{exp_name}_"
    matching_files = [
        f for f in os.listdir(results_dir) 
        if f.startswith(pattern) and f.endswith('.json')
    ]
    
    if not matching_files:
        return None
    
    # Sort by timestamp (embedded in filename)
    latest = sorted(matching_files)[-1]
    filepath = os.path.join(results_dir, latest)
    
    with open(filepath, 'r', encoding='utf-8') as f:
        return json.load(f)


def list_results(exp_name: str = None) -> list:
    """List all saved results."""
    results_dir = get_results_dir()
    
    files = [f for f in os.listdir(results_dir) if f.endswith('.json')]
    
    if exp_name:
        files = [f for f in files if f.startswith(exp_name)]
    
    return sorted(files)


def save_figure(
    fig,
    fig_name: str,
    exp_name: str = None,
    dpi: int = 300
) -> str:
    """
    Save matplotlib figure with consistent naming.
    
    Naming format: fig{N}_{description}_{timestamp}.png
    """
    results_dir = get_results_dir()
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    if exp_name:
        filename = f"{exp_name}_{fig_name}_{timestamp}.png"
    else:
        filename = f"{fig_name}_{timestamp}.png"
    
    filepath = os.path.join(results_dir, filename)
    fig.savefig(filepath, dpi=dpi, bbox_inches='tight')
    
    print(f"Figure saved: {filepath}")
    return filepath


In [ ]:
# ====================
# MODULE: civae/dag.py
# ====================
"""
DAG: Directed Acyclic Graph with Markov Blanket support
"""

import numpy as np


class DAG:
    """Directed Acyclic Graph with Markov Blanket computation."""
    
    def __init__(self, adjacency_matrix: np.ndarray):
        """
        Initialize DAG from adjacency matrix.
        
        Args:
            adjacency_matrix: [d, d] matrix where A[i,j]=1 means i→j
        """
        self.A = np.array(adjacency_matrix, dtype=np.float32)
        self.d = self.A.shape[0]
        self._validate_dag()
        self._topo_order: Optional[List[int]] = None
        self._mb_cache: Dict[int, Set[int]] = {}
        self._level_cache: Dict[int, int] = {}
    
    def _validate_dag(self):
        """Validate that the graph is a DAG (no cycles)."""
        visited = set()
        rec_stack = set()
        
        def dfs(node):
            visited.add(node)
            rec_stack.add(node)
            for child in self.children(node):
                if child not in visited:
                    if dfs(child):
                        return True
                elif child in rec_stack:
                    return True
            rec_stack.remove(node)
            return False
        
        for node in range(self.d):
            if node not in visited:
                if dfs(node):
                    raise ValueError("Graph contains a cycle, not a valid DAG")
    
    @classmethod
    def from_adjacency(cls, adj: List[List[int]]) -> 'DAG':
        """Create DAG from adjacency list."""
        return cls(np.array(adj))
    
    @classmethod
    def random_dag(cls, d: int, edge_prob: float = 0.3, seed: int = 42) -> 'DAG':
        """Generate random DAG with given density."""
        np.random.seed(seed)
        A = np.zeros((d, d))
        for i in range(d):
            for j in range(i + 1, d):  # Upper triangular for DAG
                if np.random.rand() < edge_prob:
                    A[i, j] = 1
        return cls(A)
    
    def parents(self, i: int) -> Set[int]:
        """Get parent nodes of node i."""
        return set(np.where(self.A[:, i] == 1)[0])
    
    def children(self, i: int) -> Set[int]:
        """Get child nodes of node i."""
        return set(np.where(self.A[i, :] == 1)[0])
    
    def coparents(self, i: int) -> Set[int]:
        """Get co-parents of node i (other parents of i's children)."""
        copa = set()
        for c in self.children(i):
            copa.update(self.parents(c))
        copa.discard(i)
        return copa
    
    def markov_blanket(self, i: int) -> Set[int]:
        """
        Get Markov blanket of node i.
        MB(i) = Parents(i) ∪ Children(i) ∪ CoParents(i)
        """
        if i in self._mb_cache:
            return self._mb_cache[i]
        
        mb = self.parents(i) | self.children(i) | self.coparents(i)
        self._mb_cache[i] = mb
        return mb
    
    def level(self, i: int) -> int:
        """Get topological level of node i."""
        if i in self._level_cache:
            return self._level_cache[i]
        
        parents = self.parents(i)
        if not parents:
            level = 0
        else:
            level = max(self.level(p) for p in parents) + 1
        
        self._level_cache[i] = level
        return level
    
    @property
    def topo_order(self) -> List[int]:
        """Get topological ordering of nodes."""
        if self._topo_order is not None:
            return self._topo_order
        
        in_degree = np.sum(self.A, axis=0)
        queue = deque([i for i in range(self.d) if in_degree[i] == 0])
        order = []
        
        while queue:
            node = queue.popleft()
            order.append(node)
            for child in self.children(node):
                in_degree[child] -= 1
                if in_degree[child] == 0:
                    queue.append(child)
        
        self._topo_order = order
        return order
    
    @property
    def roots(self) -> List[int]:
        """Get root nodes (no parents)."""
        return [i for i in range(self.d) if len(self.parents(i)) == 0]
    
    def is_generic(self) -> bool:
        """Check if DAG is generic (all MB unique)."""
        mbs = [frozenset(self.markov_blanket(i)) for i in range(self.d)]
        return len(set(mbs)) == self.d
    
    def __len__(self) -> int:
        return self.d
    
    def __repr__(self) -> str:
        edges = []
        for i in range(self.d):
            for j in self.children(i):
                edges.append(f"{i}→{j}")
        return f"DAG(d={self.d}, edges=[{', '.join(edges)}])"


In [ ]:
# ====================
# MODULE: civae/mb_encoder.py
# ====================
"""
Markov Blanket Encoder for C-iVAE

Per Parent-Induced Variability theory:
- MB embeddings serve as STRUCTURAL IDENTIFIERS for each node
- They ensure different nodes have different parameter functions
- This prevents degenerate solutions where two nodes could be mixed

Role in identifiability:
- Root nodes: Need environment variation (standard iVAE)
- Non-root nodes: Get variation from parents (Parent-Induced Variability)
- MB ensures parameter function uniqueness across all nodes

Non-degeneracy guarantee (A3'):
- Random Fourier Features (RFF) approximate RBF kernel feature maps
- RFF ensures that λ(u_i) - λ(u_0) vectors are likely linearly independent
- This satisfies the iVAE sufficient variability condition
"""

import numpy as np
import torch
import torch.nn as nn



class RandomFourierFeatures(nn.Module):
    """
    Random Fourier Features for RBF kernel approximation.
    
    Theory: RFF provides a finite-dimensional approximation to the RBF kernel's
    infinite-dimensional feature map. This helps ensure the non-degeneracy
    condition (A3') in the identifiability theorem.
    
    Reference: Rahimi & Recht, "Random Features for Large-Scale Kernel Machines", NeurIPS 2007
    """
    
    def __init__(self, input_dim: int, num_features: int = 256, sigma: float = 1.0):
        """
        Args:
            input_dim: Dimension of input vectors
            num_features: Number of random features (higher = better approximation)
            sigma: RBF kernel bandwidth parameter
        """
        super().__init__()
        self.num_features = num_features
        
        # Random projection matrix (frozen, not learned)
        # W ~ N(0, 1/sigma^2)
        self.register_buffer(
            'W', torch.randn(input_dim, num_features) / sigma
        )
        # Random phase shift b ~ Uniform(0, 2π)
        self.register_buffer(
            'b', torch.rand(num_features) * 2 * np.pi
        )
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Compute RFF: z(x) = sqrt(2/D) * cos(Wx + b)
        
        Args:
            x: Input tensor of shape [..., input_dim]
            
        Returns:
            RFF features of shape [..., num_features]
        """
        # Project and apply cosine
        proj = x @ self.W + self.b  # [..., num_features]
        return torch.cos(proj) * np.sqrt(2.0 / self.num_features)


class MBEncoder(nn.Module):
    """
    Markov Blanket Encoder with RFF Non-degeneracy Guarantee.
    
    Key insight: MB(i) = Pa(i) ∪ Ch(i) ∪ CoPa(i) is unique for each node
    in a generic DAG. By encoding this structural information, we create
    node-specific auxiliary variables that break rotational symmetry.
    
    Implementation details:
    1. DeepSets for permutation-invariant MB aggregation
    2. Node's own index included to handle symmetric graphs (V-structure)
    3. RFF layer to ensure non-degeneracy (A3' condition)
    """
    
    def __init__(self, num_nodes: int, embed_dim: int = 64, 
                 rff_features: int = 256, rff_sigma: float = 1.0,
                 use_rff: bool = True):
        """
        Args:
            num_nodes: Number of nodes in the DAG (d)
            embed_dim: Dimension of the MB embedding (h)
            rff_features: Number of Random Fourier Features
            rff_sigma: RBF kernel bandwidth for RFF
            use_rff: Whether to use RFF layer (disable for ablation studies)
        """
        super().__init__()
        self.num_nodes = num_nodes
        self.embed_dim = embed_dim
        self.use_rff = use_rff
        
        # Node identity embeddings (includes node's own index for symmetry breaking)
        self.node_embed = nn.Embedding(num_nodes, embed_dim)
        
        # DeepSets: φ (element-wise transform)
        self.phi = nn.Sequential(
            nn.Linear(embed_dim, embed_dim),
            nn.ReLU(),
            nn.Linear(embed_dim, embed_dim)
        )
        
        # DeepSets: ρ (aggregate transform) 
        # Input: MB aggregation + node's own embedding + [level, |Pa|, |Ch|, |MB|]
        # Note: We include node's own embedding to break symmetry in V-structures
        pre_rho_dim = embed_dim * 2 + 4  # MB agg + self embed + stats
        
        self.rho = nn.Sequential(
            nn.Linear(pre_rho_dim, embed_dim),
            nn.ReLU(),
            nn.Linear(embed_dim, embed_dim)
        )
        
        # Random Fourier Features for non-degeneracy guarantee
        if use_rff:
            self.rff = RandomFourierFeatures(embed_dim, rff_features, rff_sigma)
            self.rff_proj = nn.Linear(rff_features, embed_dim)
        
        # Empty set embedding
        self.empty_embed = nn.Parameter(torch.zeros(embed_dim))
        
        # Cache for computed embeddings (cleared on each forward)
        self._cache: Dict[int, torch.Tensor] = {}
    
    def _encode_mb(self, dag: DAG, node_i: int, device: torch.device) -> torch.Tensor:
        """
        Encode the Markov Blanket of a single node using DeepSets + RFF.
        
        Args:
            dag: The DAG structure
            node_i: Node index
            device: Device to create tensors on
            
        Returns:
            Tensor of shape [embed_dim] representing u_i
        """
        mb: Set[int] = dag.markov_blanket(node_i)
        
        # CRITICAL: Include node's own embedding to break symmetry
        # This ensures u_i ≠ u_j even for V-structure nodes A→C←B where MB(A)=MB(B) as sets
        self_idx = torch.tensor(node_i, dtype=torch.long, device=device)
        self_embed = self.node_embed(self_idx)  # [embed_dim]
        
        if len(mb) == 0:
            # Empty MB: use special embedding
            mb_agg = self.empty_embed
        else:
            # Get MB node embeddings
            mb_idx = torch.tensor(list(mb), dtype=torch.long, device=device)
            mb_embeds = self.node_embed(mb_idx)  # [|MB|, embed_dim]
            
            # DeepSets: sum of transformed embeddings
            transformed = self.phi(mb_embeds)     # [|MB|, embed_dim]
            mb_agg = transformed.sum(dim=0)       # [embed_dim]
        
        # Concatenate with structural stats
        stats = torch.tensor([
            dag.level(node_i),
            len(dag.parents(node_i)),
            len(dag.children(node_i)),
            len(mb)
        ], dtype=torch.float32, device=device)
        
        # Combine: MB aggregation + self embedding + stats
        combined = torch.cat([mb_agg, self_embed, stats])
        u_pre = self.rho(combined)  # [embed_dim]
        
        # Apply RFF for non-degeneracy guarantee
        if self.use_rff:
            rff_features = self.rff(u_pre)  # [rff_features]
            u_i = self.rff_proj(rff_features)  # [embed_dim]
        else:
            u_i = u_pre
        
        return u_i
    
    def forward(self, dag: DAG, device: torch.device) -> Dict[int, torch.Tensor]:
        """
        Compute MB embeddings for all nodes.
        
        Note: We do NOT cache here to avoid gradient graph issues during training.
        Each forward call creates fresh embeddings with proper gradients.
        
        Args:
            dag: The DAG structure
            device: Device for tensors
            
        Returns:
            Dictionary mapping node index to its MB embedding u_i
        """
        u = {}
        for i in range(dag.d):
            u[i] = self._encode_mb(dag, i, device)
        return u
    
    def compute_lambda_matrix_rank(self, dag: DAG, device: torch.device) -> Dict[str, float]:
        """
        Diagnostic function: Compute the rank and condition number of the 
        λ(u_i) difference matrix L.
        
        This is used to verify the non-degeneracy condition (A3') in practice.
        
        Returns:
            Dictionary with 'rank', 'condition_number', 'rank_ratio' metrics
        """
        u = self.forward(dag, device)
        
        # Stack all u_i into a matrix [d, embed_dim]
        u_matrix = torch.stack([u[i] for i in range(dag.d)], dim=0)
        
        # Compute difference matrix: L[l] = u[l] - u[0]
        u_0 = u_matrix[0:1, :]  # [1, embed_dim]
        L = u_matrix[1:, :] - u_0  # [d-1, embed_dim]
        
        # Compute SVD for rank analysis
        try:
            U, S, V = torch.svd(L)
            rank = (S > 1e-6).sum().item()
            condition_number = (S[0] / S[-1]).item() if S[-1] > 1e-10 else float('inf')
            rank_ratio = rank / min(L.shape[0], L.shape[1])
        except:
            rank, condition_number, rank_ratio = 0, float('inf'), 0.0
        
        return {
            'rank': rank,
            'condition_number': condition_number,
            'rank_ratio': rank_ratio,
            'matrix_shape': list(L.shape)
        }


In [ ]:
# ====================
# MODULE: civae/gnn_encoder.py
# ====================
"""
Asymmetric Structure Encoder for C-iVAE

This module implements a GNN-based structure encoder that learns intrinsic
asymmetry from DAG structure WITHOUT using explicit node indices.

Key Design Principles:
1. No explicit node index in encoding - learns asymmetry from structure
2. Uses structural positional encoding (random walk, centrality, degree)
3. Graphormer-style architecture with spatial and edge encoding
4. Random perturbation (node dropout) for robustness

Theory:
- Replaces u_i = φ(MB(i), i) with u_i = GNN(G, perturb)[i]
- Even if MB(A) = MB(B), different global positions yield different u_i
- Satisfies A6 (symmetry breaking) through learned structural features

Reference:
- Ying et al., "Do Transformers Really Perform Bad for Graph Representation?", NeurIPS 2021
- Dwivedi & Bresson, "A Generalization of Transformer Networks to Graphs", AAAI 2021
"""

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import math



class StructuralPositionalEncoding(nn.Module):
    """
    Computes structural positional encodings for each node in a DAG.
    
    Features computed:
    1. Random Walk Encoding (RWE): k-step landing probabilities
    2. Degree Features: in-degree, out-degree, normalized
    3. Centrality Features: PageRank approximation
    4. DAG-specific: topological level, distance to roots
    5. Spatial Encoding: shortest path distances (Graphormer-style)
    
    These features capture intrinsic structural asymmetry without node indices.
    """
    
    def __init__(self, num_nodes: int, embed_dim: int, 
                 rw_steps: int = 8, use_centrality: bool = True):
        """
        Args:
            num_nodes: Number of nodes in DAG (d)
            embed_dim: Output embedding dimension
            rw_steps: Number of random walk steps for RWE
            use_centrality: Whether to compute centrality features
        """
        super().__init__()
        self.num_nodes = num_nodes
        self.rw_steps = rw_steps
        self.use_centrality = use_centrality
        
        # Feature dimensions:
        # - RWE: rw_steps (landing probs for each step)
        # - Degree: 4 (in, out, total, normalized)
        # - Centrality: 1 (PageRank)
        # - DAG: 3 (level, min_dist_to_root, max_dist_to_root)
        raw_dim = rw_steps + 4 + (1 if use_centrality else 0) + 3
        
        self.proj = nn.Sequential(
            nn.Linear(raw_dim, embed_dim),
            nn.LayerNorm(embed_dim),
            nn.ReLU(),
            nn.Linear(embed_dim, embed_dim),
            nn.LayerNorm(embed_dim)
        )
        
        # Learnable spatial encoding for shortest path distances
        # Graphormer uses learnable biases for each distance
        self.max_dist = 10  # Cap distances at this value
        self.spatial_encoder = nn.Embedding(self.max_dist + 2, 1)  # +2 for self and unreachable
        
    def _compute_random_walk_encoding(self, adj: torch.Tensor) -> torch.Tensor:
        """
        Compute random walk landing probabilities.
        
        Args:
            adj: Adjacency matrix [d, d]
            
        Returns:
            RWE matrix [d, rw_steps] - landing prob at each step
        """
        d = adj.size(0)
        device = adj.device
        
        # Compute transition matrix (row-normalized for random walk)
        # For DAG, we use symmetric version for undirected walks
        adj_sym = adj + adj.t()
        deg = adj_sym.sum(dim=1, keepdim=True).clamp(min=1)
        trans = adj_sym / deg  # [d, d]
        
        rwe = []
        p = torch.eye(d, device=device)  # Start at each node
        
        for _ in range(self.rw_steps):
            p = p @ trans
            rwe.append(p.diag())  # Landing probability at self
            
        return torch.stack(rwe, dim=1)  # [d, rw_steps]
    
    def _compute_degree_features(self, adj: torch.Tensor) -> torch.Tensor:
        """
        Compute degree-based features.
        
        Args:
            adj: Adjacency matrix [d, d]
            
        Returns:
            Degree features [d, 4]
        """
        in_deg = adj.sum(dim=0)   # Column sum
        out_deg = adj.sum(dim=1)  # Row sum
        total_deg = in_deg + out_deg
        
        # Normalized by max degree
        max_deg = total_deg.max().clamp(min=1)
        norm_deg = total_deg / max_deg
        
        return torch.stack([in_deg, out_deg, total_deg, norm_deg], dim=1)
    
    def _compute_pagerank(self, adj: torch.Tensor, 
                          damping: float = 0.85, 
                          iterations: int = 20) -> torch.Tensor:
        """
        Approximate PageRank centrality via power iteration.
        
        Args:
            adj: Adjacency matrix [d, d]
            damping: Damping factor
            iterations: Number of power iterations
            
        Returns:
            PageRank scores [d, 1]
        """
        d = adj.size(0)
        device = adj.device
        
        # Use symmetric adjacency for undirected centrality
        adj_sym = adj + adj.t()
        deg = adj_sym.sum(dim=1, keepdim=True).clamp(min=1)
        trans = adj_sym / deg
        
        # Initialize uniform
        pr = torch.ones(d, device=device) / d
        
        for _ in range(iterations):
            pr = (1 - damping) / d + damping * (trans.t() @ pr)
            
        return pr.unsqueeze(1)  # [d, 1]
    
    def _compute_dag_features(self, dag: DAG, device: torch.device) -> torch.Tensor:
        """
        Compute DAG-specific structural features.
        
        Args:
            dag: The DAG object
            device: Device for tensors
            
        Returns:
            DAG features [d, 3]
        """
        levels = torch.tensor([dag.level(i) for i in range(dag.d)], 
                              dtype=torch.float32, device=device)
        
        # Distance to roots (BFS from each root)
        roots = dag.roots
        if len(roots) == 0:
            min_dist = torch.zeros(dag.d, device=device)
            max_dist = torch.zeros(dag.d, device=device)
        else:
            adj = torch.from_numpy(dag.A).float().to(device)
            adj_t = adj.t()  # For reverse traversal
            
            distances = []
            for root in roots:
                # BFS from root
                dist = torch.full((dag.d,), float('inf'), device=device)
                dist[root] = 0
                for _ in range(dag.d):
                    # Propagate: dist[child] = min(dist[child], dist[parent] + 1)
                    new_dist = dist.unsqueeze(0) + adj  # [d, d]
                    new_dist = torch.where(adj > 0, new_dist, 
                                          torch.full_like(new_dist, float('inf')))
                    dist = torch.min(dist, new_dist.min(dim=0)[0])
                distances.append(dist)
            
            dist_stack = torch.stack(distances, dim=0)  # [num_roots, d]
            min_dist = dist_stack.min(dim=0)[0]
            max_dist = torch.where(dist_stack == float('inf'), 
                                   torch.zeros_like(dist_stack), dist_stack).max(dim=0)[0]
            
            # Replace inf with max level + 1
            max_level = levels.max() + 1
            min_dist = torch.where(min_dist == float('inf'), max_level, min_dist)
            max_dist = torch.where(max_dist == float('inf'), max_level, max_dist)
        
        # Normalize by max level
        max_level = levels.max().clamp(min=1)
        levels_norm = levels / max_level
        min_dist_norm = min_dist / max_level
        max_dist_norm = max_dist / max_level
        
        return torch.stack([levels_norm, min_dist_norm, max_dist_norm], dim=1)
    
    def _compute_shortest_path_matrix(self, adj: torch.Tensor) -> torch.Tensor:
        """
        Compute all-pairs shortest path distances using Floyd-Warshall.
        This is used for Graphormer's spatial encoding.
        
        Args:
            adj: Adjacency matrix [d, d]
            
        Returns:
            Distance matrix [d, d] (capped at max_dist, unreachable = max_dist + 1)
        """
        d = adj.size(0)
        device = adj.device
        
        # Use undirected graph for spatial encoding
        adj_sym = (adj + adj.t()).clamp(max=1)
        
        # Initialize distance matrix
        inf = float('inf')
        dist = torch.where(adj_sym > 0, torch.ones_like(adj_sym),
                          torch.full_like(adj_sym, inf))
        dist.fill_diagonal_(0)
        
        # Floyd-Warshall
        for k in range(d):
            dist = torch.min(dist, dist[:, k:k+1] + dist[k:k+1, :])
        
        # Cap distances
        dist = torch.where(dist == inf, 
                          torch.full_like(dist, self.max_dist + 1),
                          dist.clamp(max=self.max_dist))
        
        return dist.long()
    
    def forward(self, dag: DAG, device: torch.device) -> Tuple[torch.Tensor, torch.Tensor]:
        """
        Compute positional encodings for all nodes.
        
        Args:
            dag: The DAG structure
            device: Device for tensors
            
        Returns:
            node_pe: Node positional encodings [d, embed_dim]
            spatial_bias: Spatial attention bias [d, d]
        """
        adj = torch.from_numpy(dag.A).float().to(device)
        
        # Compute structural features
        rwe = self._compute_random_walk_encoding(adj)      # [d, rw_steps]
        deg = self._compute_degree_features(adj)           # [d, 4]
        dag_feat = self._compute_dag_features(dag, device) # [d, 3]
        
        features = [rwe, deg, dag_feat]
        
        if self.use_centrality:
            pr = self._compute_pagerank(adj)               # [d, 1]
            features.append(pr)
        
        raw_pe = torch.cat(features, dim=1)  # [d, raw_dim]
        node_pe = self.proj(raw_pe)          # [d, embed_dim]
        
        # Compute spatial encoding (Graphormer-style)
        dist_matrix = self._compute_shortest_path_matrix(adj)  # [d, d]
        spatial_bias = self.spatial_encoder(dist_matrix).squeeze(-1)  # [d, d]
        
        return node_pe, spatial_bias


class GraphTransformerLayer(nn.Module):
    """
    Graphormer-style Transformer layer with structural biases.
    
    Key differences from standard Transformer:
    1. Spatial encoding bias in attention (based on shortest path distance)
    2. Edge encoding can be added (not used for DAG structure learning)
    3. Layer-wise learnable biases for centrality (optional)
    """
    
    def __init__(self, embed_dim: int, num_heads: int = 4, 
                 ff_dim: int = None, dropout: float = 0.1):
        super().__init__()
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.head_dim = embed_dim // num_heads
        
        if ff_dim is None:
            ff_dim = embed_dim * 4
        
        # Multi-head self-attention
        self.q_proj = nn.Linear(embed_dim, embed_dim)
        self.k_proj = nn.Linear(embed_dim, embed_dim)
        self.v_proj = nn.Linear(embed_dim, embed_dim)
        self.out_proj = nn.Linear(embed_dim, embed_dim)
        
        # Feed-forward network
        self.ffn = nn.Sequential(
            nn.Linear(embed_dim, ff_dim),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(ff_dim, embed_dim),
            nn.Dropout(dropout)
        )
        
        # Layer norms (pre-norm architecture)
        self.norm1 = nn.LayerNorm(embed_dim)
        self.norm2 = nn.LayerNorm(embed_dim)
        
        self.dropout = nn.Dropout(dropout)
        self.attn_scale = self.head_dim ** -0.5
        
    def forward(self, x: torch.Tensor, 
                spatial_bias: Optional[torch.Tensor] = None) -> torch.Tensor:
        """
        Forward pass with optional spatial bias.
        
        Args:
            x: Node embeddings [d, embed_dim]
            spatial_bias: Spatial attention bias [d, d]
            
        Returns:
            Updated node embeddings [d, embed_dim]
        """
        d = x.size(0)
        
        # Pre-norm self-attention
        h = self.norm1(x)
        
        # Multi-head attention
        Q = self.q_proj(h).view(d, self.num_heads, self.head_dim).transpose(0, 1)  # [heads, d, head_dim]
        K = self.k_proj(h).view(d, self.num_heads, self.head_dim).transpose(0, 1)
        V = self.v_proj(h).view(d, self.num_heads, self.head_dim).transpose(0, 1)
        
        # Attention scores
        attn = torch.bmm(Q, K.transpose(1, 2)) * self.attn_scale  # [heads, d, d]
        
        # Add spatial bias (Graphormer-style)
        if spatial_bias is not None:
            attn = attn + spatial_bias.unsqueeze(0)  # Broadcast over heads
        
        attn = F.softmax(attn, dim=-1)
        attn = self.dropout(attn)
        
        # Apply attention to values
        out = torch.bmm(attn, V)  # [heads, d, head_dim]
        out = out.transpose(0, 1).contiguous().view(d, self.embed_dim)  # [d, embed_dim]
        out = self.out_proj(out)
        
        # Residual connection
        x = x + self.dropout(out)
        
        # Pre-norm FFN
        x = x + self.ffn(self.norm2(x))
        
        return x


class AsymmetricStructureEncoder(nn.Module):
    """
    GNN-based Asymmetric Structure Encoder for C-iVAE.
    
    This encoder learns to generate node-specific auxiliary variables u_i
    WITHOUT using explicit node indices. It relies on:
    
    1. Structural Positional Encoding: captures global position via
       random walks, centrality, and DAG-specific features
    2. Graph Transformer: propagates and refines structural information
    3. Random Perturbation: node dropout during training for robustness
    
    Theory:
    - Even if MB(A) = MB(B) (same local structure), different global
      positions in the DAG lead to different u_i
    - This satisfies the symmetry-breaking requirement (A6) without
      relying on node indices as "weak supervision"
    
    Example:
        >>> dag = DAG.random_dag(10, edge_prob=0.3)
        >>> encoder = AsymmetricStructureEncoder(dag.d, embed_dim=64)
        >>> u = encoder(dag, device)  # Returns {i: u_i tensor}
    """
    
    def __init__(self, num_nodes: int, embed_dim: int = 64,
                 num_layers: int = 3, num_heads: int = 4,
                 dropout: float = 0.1, node_dropout: float = 0.1,
                 rw_steps: int = 8, use_centrality: bool = True):
        """
        Args:
            num_nodes: Number of nodes in the DAG (d)
            embed_dim: Dimension of node embeddings (u_dim)
            num_layers: Number of Graph Transformer layers
            num_heads: Number of attention heads per layer
            dropout: Dropout rate in Transformer layers
            node_dropout: Probability of dropping node features during training
            rw_steps: Number of random walk steps for positional encoding
            use_centrality: Whether to use centrality features
        """
        super().__init__()
        self.num_nodes = num_nodes
        self.embed_dim = embed_dim
        self.node_dropout = node_dropout
        
        # Structural positional encoding
        self.pos_encoder = StructuralPositionalEncoding(
            num_nodes, embed_dim, rw_steps, use_centrality
        )
        
        # Graph Transformer layers
        self.layers = nn.ModuleList([
            GraphTransformerLayer(embed_dim, num_heads, dropout=dropout)
            for _ in range(num_layers)
        ])
        
        # Final projection (with RFF-like non-linearity for diversity)
        self.output_proj = nn.Sequential(
            nn.Linear(embed_dim, embed_dim * 2),
            nn.GELU(),
            nn.Linear(embed_dim * 2, embed_dim),
            nn.LayerNorm(embed_dim)
        )
        
    def forward(self, dag: DAG, device: torch.device) -> Dict[int, torch.Tensor]:
        """
        Compute structural embeddings for all nodes.
        
        Args:
            dag: The DAG structure
            device: Device for tensors
            
        Returns:
            Dictionary mapping node index i to embedding u_i [embed_dim]
        """
        # Get positional encodings and spatial bias
        node_pe, spatial_bias = self.pos_encoder(dag, device)  # [d, embed_dim], [d, d]
        
        # Apply node dropout during training (random perturbation)
        if self.training and self.node_dropout > 0:
            mask = torch.bernoulli(
                torch.full((dag.d, 1), 1 - self.node_dropout, device=device)
            )
            # Rescale to maintain expected value
            node_pe = node_pe * mask / (1 - self.node_dropout)
        
        # Pass through Graph Transformer layers
        h = node_pe
        for layer in self.layers:
            h = layer(h, spatial_bias)
        
        # Project to output space
        u_matrix = self.output_proj(h)  # [d, embed_dim]
        
        # Return as dictionary (compatible with MBEncoder interface)
        return {i: u_matrix[i] for i in range(dag.d)}
    
    def compute_lambda_matrix_rank(self, dag: DAG, device: torch.device) -> Dict[str, float]:
        """
        Diagnostic: Compute rank of the λ difference matrix.
        
        This verifies the non-degeneracy condition (A3') - that the
        u_i vectors are sufficiently diverse for identifiability.
        
        Returns:
            Dictionary with 'rank', 'condition_number', 'rank_ratio'
        """
        self.eval()
        with torch.no_grad():
            u = self.forward(dag, device)
            u_matrix = torch.stack([u[i] for i in range(dag.d)], dim=0)
            
            # Compute L[l] = u[l] - u[0]
            L = u_matrix[1:] - u_matrix[0:1]  # [d-1, embed_dim]
            
            try:
                _, S, _ = torch.svd(L)
                rank = (S > 1e-6).sum().item()
                cond = (S[0] / S[-1]).item() if S[-1] > 1e-10 else float('inf')
                rank_ratio = rank / min(L.shape)
            except:
                rank, cond, rank_ratio = 0, float('inf'), 0.0
        
        self.train()
        return {
            'rank': rank,
            'condition_number': cond,
            'rank_ratio': rank_ratio,
            'matrix_shape': list(L.shape)
        }
    
    def get_pairwise_distances(self, dag: DAG, device: torch.device) -> torch.Tensor:
        """
        Diagnostic: Compute pairwise distances between node embeddings.
        
        Useful for visualizing how well the encoder distinguishes nodes.
        
        Returns:
            Distance matrix [d, d]
        """
        self.eval()
        with torch.no_grad():
            u = self.forward(dag, device)
            u_matrix = torch.stack([u[i] for i in range(dag.d)], dim=0)
            dist = torch.cdist(u_matrix.unsqueeze(0), u_matrix.unsqueeze(0)).squeeze()
        self.train()
        return dist


In [ ]:
# ====================
# MODULE: civae/prior.py
# ====================
"""
Causal Prior Network
p(z_i | Pa(z_i), u_i)
"""

import torch
import torch.nn as nn
import torch.nn.functional as F

class CausalPrior(nn.Module):
    """
    Causal prior with root/non-root distinction.
    - Root nodes: p(z_i | u_i, e) - depends on environment
    - Non-root nodes: p(z_i | Pa(z_i), u_i) - depends on parents (Parent-Induced Variability)
    """
    
    def __init__(self, z_dim: int, u_dim: int, num_envs: int, hidden_dim: int = 128):
        super().__init__()
        self.z_dim = z_dim
        self.u_dim = u_dim
        
        self.env_embed = nn.Embedding(num_envs, hidden_dim // 4)
        
        # Root prior: p(z_i | u_i, e) - needs environment for identifiability
        self.root_net = nn.Sequential(
            nn.Linear(u_dim + hidden_dim // 4, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU()
        )
        self.root_mu = nn.Linear(hidden_dim, z_dim)
        self.root_logvar = nn.Linear(hidden_dim, z_dim)
        
        # Non-root prior: p(z_i | Pa(z_i), u_i)
        # Parent aggregation via attention
        # Input to attention is now Concat(z_parent, u_parent)
        self.parent_key = nn.Linear(z_dim + u_dim, hidden_dim)
        self.parent_query = nn.Linear(u_dim, hidden_dim)
        self.parent_value = nn.Linear(z_dim + u_dim, hidden_dim)
        
        self.cond_net = nn.Sequential(
            nn.Linear(hidden_dim + u_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU()
        )
        self.cond_mu = nn.Linear(hidden_dim, z_dim)
        self.cond_logvar = nn.Linear(hidden_dim, z_dim)
    
    def forward(self, parent_z: List[torch.Tensor], u_parents: List[torch.Tensor],
                u_i: torch.Tensor, env: torch.Tensor, is_root: bool) -> Tuple[torch.Tensor, torch.Tensor]:
        """
        Compute prior parameters for node i.
        
        Args:
            parent_z: List of parent z tensors, each [batch, z_dim]
            u_parents: List of parent structural embeddings, each [u_dim] or [batch, u_dim]
            u_i: MB encoding [u_dim] (structural, same for all samples)
            env: Environment index [batch]
            is_root: Whether this is a root node
        """
        batch_size = u_i.size(0)
        device = u_i.device
        
        # Expand u_i to batch if needed
        if u_i.dim() == 1:
            u_i_batch = u_i.unsqueeze(0).expand(batch_size, -1)
        else:
            u_i_batch = u_i
            
        if is_root:
            # Root node: use environment for variation
            env_emb = self.env_embed(env)
            h = self.root_net(torch.cat([u_i_batch, env_emb], dim=-1))
            return self.root_mu(h), self.root_logvar(h)
        else:
            # Non-root: aggregate parents via attention
            if not parent_z:
                # Should not happen for non-root check
                return torch.zeros(batch_size, self.z_dim, device=device), \
                       torch.zeros(batch_size, self.z_dim, device=device)
            else:
                # Stack parents: [batch, num_parents, z_dim]
                parent_stack_z = torch.stack(parent_z, dim=1)
                
                # Stack parent embeddings: [batch, num_parents, u_dim]
                u_parents_expanded = []
                for up in u_parents:
                    if up.dim() == 1:
                        u_parents_expanded.append(up.unsqueeze(0).expand(batch_size, -1))
                    else:
                        u_parents_expanded.append(up)
                parent_stack_u = torch.stack(u_parents_expanded, dim=1)
                
                # Input to Attention: Concat(z, u) to identify WHICH parent implies WHAT value
                # This is critical for structure learning
                parent_stack = torch.cat([parent_stack_z, parent_stack_u], dim=-1)
                
                # Update attention layers dimensions dynamically if needed, 
                # but here we assume fixed sizes defined in init.
                # Wait, input dim to Key/Value changed from z_dim to z_dim + u_dim
                # We need to update __init__ as well.
                
                K = self.parent_key(parent_stack)     # [batch, num_parents, hidden]
                Q = self.parent_query(u_i_batch).unsqueeze(1)  # [batch, 1, hidden]
                V = self.parent_value(parent_stack)   # [batch, num_parents, hidden]
                
                attn = torch.softmax(torch.bmm(Q, K.transpose(1, 2)) / (K.size(-1) ** 0.5), dim=-1)
                parent_agg = torch.bmm(attn, V).squeeze(1)  # [batch, hidden]
            
            # Combine with u_i
            h = self.cond_net(torch.cat([parent_agg, u_i_batch], dim=-1))
            return self.cond_mu(h), self.cond_logvar(h)


In [ ]:
# ====================
# MODULE: civae/model.py
# ====================
"""
C-iVAE with Parent-Induced Variability

Key theoretical insight:
- Root nodes: Need environment variation for identifiability
- Non-root nodes: Get variation from parent values automatically

This reduces required environments from O(dk) to O(rk), where r is #roots.

Structure Encoder Options:
- 'gnn': Asymmetric GNN encoder (Graphormer-style) - learns intrinsic asymmetry
         without explicit node indices. Recommended for theoretical purity.
- 'mb': Original MB encoder with node index embedding. More stable but uses
        node index as structural information.
"""

import torch
import torch.nn as nn
import torch.nn.functional as F



class CausalEncoder(nn.Module):
    """Causal encoder: q(z_i | x, u_i, e)"""
    
    def __init__(self, input_dim: int, z_dim: int, num_nodes: int,
                 u_dim: int, num_envs: int, hidden_dim: int = 256):
        super().__init__()
        self.z_dim = z_dim
        self.num_nodes = num_nodes
        self.u_dim = u_dim
        
        self.env_embed = nn.Embedding(num_envs, hidden_dim // 4)
        
        self.feat_net = nn.Sequential(
            nn.Linear(input_dim + hidden_dim // 4, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU()
        )
        
        self.node_nets = nn.ModuleList([
            nn.Sequential(
                nn.Linear(hidden_dim + u_dim + z_dim * num_nodes, hidden_dim // 2),
                nn.ReLU()
            ) for _ in range(num_nodes)
        ])
        
        self.mu_heads = nn.ModuleList([nn.Linear(hidden_dim // 2, z_dim) for _ in range(num_nodes)])
        self.logvar_heads = nn.ModuleList([nn.Linear(hidden_dim // 2, z_dim) for _ in range(num_nodes)])
    
    def forward(self, x: torch.Tensor, u: Dict[int, torch.Tensor], 
                env: torch.Tensor, dag: 'DAG',
                topo_order: List[int]) -> Tuple[Dict, Dict, Dict]:
        batch_size = x.size(0)
        device = x.device
        
        env_emb = self.env_embed(env)
        feat = self.feat_net(torch.cat([x, env_emb], dim=-1))
        
        z_samples, z_means, z_logvars = {}, {}, {}
        
        for i in topo_order:
            u_i = u[i].unsqueeze(0).expand(batch_size, -1)
            
            parent_indices = dag.parents(i)
            if len(parent_indices) > 0:
                parent_z = torch.cat([z_samples[j] for j in sorted(parent_indices)], dim=-1)
            else:
                parent_z = torch.zeros(batch_size, 0, device=device)
            
            pad_size = self.z_dim * self.num_nodes - parent_z.size(-1)
            parent_z_padded = F.pad(parent_z, (0, pad_size))
            
            h = self.node_nets[i](torch.cat([feat, u_i, parent_z_padded], dim=-1))
            mu_i = self.mu_heads[i](h)
            logvar_i = self.logvar_heads[i](h)
            
            std = torch.exp(0.5 * logvar_i)
            z_samples[i] = mu_i + torch.randn_like(std) * std
            z_means[i] = mu_i
            z_logvars[i] = logvar_i
        
        return z_samples, z_means, z_logvars


class Decoder(nn.Module):
    def __init__(self, z_total_dim: int, output_dim: int, hidden_dim: int = 256):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(z_total_dim, hidden_dim), nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim), nn.ReLU(),
            nn.Linear(hidden_dim, output_dim)
        )
    
    def forward(self, z: torch.Tensor) -> torch.Tensor:
        return self.net(z)


class CausalIVAE(nn.Module):
    """
    C-iVAE with Parent-Induced Variability.
    
    Supports three types of variables:
    1. DAG root nodes: p(z_i | u_i, e) - needs environment variation
    2. DAG non-root nodes: p(z_i | Pa(z_i), u_i) - gets variation from parents
    3. Independent variables: p(z_j | e) - not in DAG, uses iVAE-style prior
    
    This allows partial structure knowledge - only model variables with known
    causal relationships, while others are treated as independent.
    
    Args:
        input_dim: Dimension of input data x
        z_dim: Dimension of each latent factor z_i
        dag: The DAG structure (only for structured variables)
        num_envs: Number of environments (for root/independent node identifiability)
        u_dim: Dimension of structural embedding u_i
        hidden_dim: Hidden layer dimension
        encoder_type: Structure encoder type ('gnn' or 'mb')
        independent_vars: List of variable indices NOT in DAG (will be treated as
                         independent roots with iVAE-style prior)
    """
    
    def __init__(self, input_dim: int, z_dim: int, dag: DAG,
                 num_envs: int = 1, u_dim: int = 64, hidden_dim: int = 256,
                 encoder_type: Literal['gnn', 'mb'] = 'gnn',
                 independent_vars: Optional[List[int]] = None):
        super().__init__()
        self.dag = dag
        self.z_dim = z_dim
        self.u_dim = u_dim
        self.num_envs = num_envs
        self.encoder_type = encoder_type
        
        # Independent variables (not in DAG, treated as iVAE-style roots)
        self.independent_vars = set(independent_vars) if independent_vars else set()
        
        # Total number of variables = DAG nodes + independent vars
        self.d = dag.d + len(self.independent_vars)
        
        # Root nodes = DAG roots + independent variables
        # All need environment variation for identifiability
        self.roots = set(dag.roots) | self.independent_vars
        
        # Structure encoder: generates u_i for each node in DAG
        if encoder_type == 'gnn':
            self.structure_encoder = AsymmetricStructureEncoder(dag.d, u_dim)
        else:
            self.structure_encoder = MBEncoder(dag.d, u_dim)
        
        # Fixed embedding for independent variables (no structural info)
        if self.independent_vars:
            self.ind_embed = nn.Embedding(len(self.independent_vars), u_dim)
            self._ind_var_to_idx = {v: i for i, v in enumerate(sorted(self.independent_vars))}
        
        self.encoder = CausalEncoder(input_dim, z_dim, self.d, u_dim, num_envs, hidden_dim)
        self.prior = CausalPrior(z_dim, u_dim, num_envs, hidden_dim)
        self.decoder = Decoder(z_dim * self.d, input_dim, hidden_dim)
    
    def _get_u(self, device: torch.device) -> Dict[int, torch.Tensor]:
        """
        Get structural embeddings u_i for all nodes.
        - DAG nodes: from structure encoder (MB/GNN)
        - Independent nodes: from learned embedding
        """
        u = self.structure_encoder(self.dag, device)
        
        # Add embeddings for independent variables
        if self.independent_vars:
            for var_idx in self.independent_vars:
                local_idx = self._ind_var_to_idx[var_idx]
                u[var_idx] = self.ind_embed(torch.tensor(local_idx, device=device))
        
        return u
    
    def _get_topo_order(self) -> List[int]:
        """Get topological order including independent variables."""
        # Independent variables can be sampled first (no parents)
        return list(sorted(self.independent_vars)) + self.dag.topo_order
    
    def forward(self, x: torch.Tensor, env: Optional[torch.Tensor] = None
                ) -> Tuple[torch.Tensor, Dict, Dict, Dict]:
        device = x.device
        if env is None:
            env = torch.zeros(x.size(0), dtype=torch.long, device=device)
        
        u = self._get_u(device)
        topo_order = self._get_topo_order()
        z_samples, z_means, z_logvars = self.encoder(x, u, env, self.dag, topo_order)
        z_concat = torch.cat([z_samples[i] for i in range(self.d)], dim=-1)
        x_recon = self.decoder(z_concat)
        self._current_env = env
        
        return x_recon, z_samples, z_means, z_logvars
    
    def compute_loss(self, x: torch.Tensor, x_recon: torch.Tensor,
                     z_samples: Dict, z_means: Dict, z_logvars: Dict,
                     env: Optional[torch.Tensor] = None) -> Dict[str, torch.Tensor]:
        batch_size = x.size(0)
        device = x.device
        
        if env is None:
            env = getattr(self, '_current_env', torch.zeros(batch_size, dtype=torch.long, device=device))
        
        u = self._get_u(device)
        recon_loss = F.mse_loss(x_recon, x, reduction='sum') / batch_size
        
        kl_loss = 0
        for i in self._get_topo_order():
            # Independent variables have no parents
            if i in self.independent_vars:
                parent_z = []
                parent_u = []
            else:
                parents = sorted(self.dag.parents(i))
                parent_z = [z_samples[j] for j in parents]
                parent_u = [u[j] for j in parents]
            
            u_i = u[i].unsqueeze(0).expand(batch_size, -1)
            is_root = (i in self.roots)
            
            prior_mu, prior_logvar = self.prior(parent_z, parent_u, u_i, env, is_root)
            kl_i = self._kl_gaussian(z_means[i], z_logvars[i], prior_mu, prior_logvar)
            kl_loss = kl_loss + kl_i.sum() / batch_size
        
        return {'loss': recon_loss + kl_loss, 'recon_loss': recon_loss, 'kl_loss': kl_loss}
    
    def _kl_gaussian(self, mu1, logvar1, mu2, logvar2):
        var1, var2 = torch.exp(logvar1), torch.exp(logvar2)
        return 0.5 * (logvar2 - logvar1 + var1 / var2 + (mu1 - mu2) ** 2 / var2 - 1).sum(dim=-1)
    
    @torch.no_grad()
    def sample(self, n_samples: int, env_idx: int = 0, device: torch.device = None) -> torch.Tensor:
        if device is None:
            device = next(self.parameters()).device
        
        u = self._get_u(device)
        env = torch.full((n_samples,), env_idx, dtype=torch.long, device=device)
        
        z = {}
        for i in self._get_topo_order():
            # Independent variables have no parents
            if i in self.independent_vars:
                parent_z = []
                parent_u = []
            else:
                parents = sorted(self.dag.parents(i))
                parent_z = [z[j] for j in parents]
                parent_u = [u[j] for j in parents]
            
            u_i = u[i].unsqueeze(0).expand(n_samples, -1)
            is_root = (i in self.roots)
            
            prior_mu, prior_logvar = self.prior(parent_z, parent_u, u_i, env, is_root)
            std = torch.exp(0.5 * prior_logvar)
            z[i] = prior_mu + torch.randn_like(std) * std
        
        z_concat = torch.cat([z[i] for i in range(self.d)], dim=-1)
        return self.decoder(z_concat)



In [ ]:
# ====================
# MODULE: civae/trainer.py
# ====================
"""
Trainer for C-iVAE with MB + Multi-Environment Identifiability
"""

import torch
import torch.optim as optim



class Trainer:
    """Trainer for C-iVAE model with multi-environment data."""
    
    def __init__(self, model: CausalIVAE, lr: float = 1e-3, 
                 device: str = 'cuda' if torch.cuda.is_available() else 'cpu'):
        self.model = model.to(device)
        self.device = device
        self.optimizer = optim.Adam(model.parameters(), lr=lr)
        self.history = {'loss': [], 'recon_loss': [], 'kl_loss': []}
    
    def train_epoch(self, dataloader: DataLoader) -> Dict[str, float]:
        """Train for one epoch."""
        self.model.train()
        
        total_loss = 0
        total_recon = 0
        total_kl = 0
        n_batches = 0
        
        for batch in dataloader:
            # Expect (x, env) tuples from multi-environment data
            if isinstance(batch, (list, tuple)) and len(batch) >= 2:
                x, env = batch[0], batch[1]
            else:
                # Fallback: single-env mode
                x = batch[0] if isinstance(batch, (list, tuple)) else batch
                env = None
            
            x = x.to(self.device).float()
            if env is not None:
                env = env.to(self.device).long()
            
            self.optimizer.zero_grad()
            
            # C-iVAE forward with env
            x_recon, z_samples, z_means, z_logvars = self.model(x, env)
            losses = self.model.compute_loss(x, x_recon, z_samples, z_means, z_logvars, env)
            
            losses['loss'].backward()
            self.optimizer.step()
            
            total_loss += losses['loss'].item()
            total_recon += losses['recon_loss'].item()
            total_kl += losses['kl_loss'].item()
            n_batches += 1
        
        return {
            'loss': total_loss / n_batches,
            'recon_loss': total_recon / n_batches,
            'kl_loss': total_kl / n_batches
        }
    
    def fit(self, dataloader: DataLoader, epochs: int = 100, 
            verbose: bool = True) -> Dict[str, list]:
        """
        Train the model.
        
        Args:
            dataloader: Training data (should yield (x, env) tuples)
            epochs: Number of epochs
            verbose: Show progress bar
            
        Returns:
            Training history
        """
        iterator = tqdm(range(epochs), desc='Training C-iVAE') if verbose else range(epochs)
        
        for epoch in iterator:
            metrics = self.train_epoch(dataloader)
            
            self.history['loss'].append(metrics['loss'])
            self.history['recon_loss'].append(metrics['recon_loss'])
            self.history['kl_loss'].append(metrics['kl_loss'])
            
            if verbose:
                iterator.set_postfix(loss=metrics['loss'], 
                                     recon=metrics['recon_loss'],
                                     kl=metrics['kl_loss'])
        
        return self.history


In [ ]:
# ====================
# MODULE: baselines/linear_baselines.py
# ====================
"""
PCA and FastICA Baselines

Linear methods as lower bounds for comparison with iVAE/C-iVAE.
Based on iVAE paper (Khemakhem et al., 2020) experiments.
"""

import numpy as np


class PCABaseline:
    """PCA baseline - linear dimensionality reduction."""
    
    def __init__(self, n_components: int):
        self.n_components = n_components
        self.pca = PCA(n_components=n_components)
        self.scaler = StandardScaler()
    
    def fit(self, X: np.ndarray):
        X_scaled = self.scaler.fit_transform(X)
        self.pca.fit(X_scaled)
        return self
    
    def transform(self, X: np.ndarray) -> np.ndarray:
        X_scaled = self.scaler.transform(X)
        return self.pca.transform(X_scaled)
    
    def fit_transform(self, X: np.ndarray) -> np.ndarray:
        X_scaled = self.scaler.fit_transform(X)
        return self.pca.fit_transform(X_scaled)


class FastICABaseline:
    """FastICA baseline - linear ICA."""
    
    def __init__(self, n_components: int, max_iter: int = 500, random_state: int = 42):
        self.n_components = n_components
        self.ica = FastICA(n_components=n_components, max_iter=max_iter, 
                          random_state=random_state, whiten='unit-variance')
        self.scaler = StandardScaler()
    
    def fit(self, X: np.ndarray):
        X_scaled = self.scaler.fit_transform(X)
        try:
            self.ica.fit(X_scaled)
        except:
            # Fallback if ICA doesn't converge
            pass
        return self
    
    def transform(self, X: np.ndarray) -> np.ndarray:
        X_scaled = self.scaler.transform(X)
        try:
            return self.ica.transform(X_scaled)
        except:
            # Fallback to PCA if ICA fails
            return PCA(n_components=self.n_components).fit_transform(X_scaled)
    
    def fit_transform(self, X: np.ndarray) -> np.ndarray:
        X_scaled = self.scaler.fit_transform(X)
        try:
            return self.ica.fit_transform(X_scaled)
        except:
            return PCA(n_components=self.n_components).fit_transform(X_scaled)


def train_pca(x_train: np.ndarray, d: int) -> PCABaseline:
    """Train PCA baseline."""
    model = PCABaseline(n_components=d)
    model.fit(x_train)
    return model


def train_ica(x_train: np.ndarray, d: int) -> FastICABaseline:
    """Train FastICA baseline."""
    model = FastICABaseline(n_components=d)
    model.fit(x_train)
    return model


def get_latent_pca(model: PCABaseline, x: np.ndarray) -> np.ndarray:
    """Get PCA latent representation."""
    return model.transform(x)


def get_latent_ica(model: FastICABaseline, x: np.ndarray) -> np.ndarray:
    """Get ICA latent representation."""
    return model.transform(x)


In [ ]:
# ====================
# MODULE: baselines/vae.py
# ====================
"""
Simple VAE baseline for comparison
"""

import torch
import torch.nn as nn
import torch.nn.functional as F


class SimpleVAE(nn.Module):
    """Standard VAE without causal structure."""
    
    def __init__(self, input_dim: int, latent_dim: int, hidden_dim: int = 256):
        super().__init__()
        self.latent_dim = latent_dim
        self.input_dim = input_dim
        
        # Encoder
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU()
        )
        self.fc_mu = nn.Linear(hidden_dim, latent_dim)
        self.fc_logvar = nn.Linear(hidden_dim, latent_dim)
        
        # Decoder
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, input_dim)
        )
    
    def encode(self, x: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor]:
        h = self.encoder(x)
        return self.fc_mu(h), self.fc_logvar(h)
    
    def reparameterize(self, mu: torch.Tensor, logvar: torch.Tensor) -> torch.Tensor:
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std
    
    def decode(self, z: torch.Tensor) -> torch.Tensor:
        return self.decoder(z)
    
    def forward(self, x: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor, torch.Tensor]:
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        x_recon = self.decode(z)
        return x_recon, mu, logvar
    
    def compute_loss(self, x: torch.Tensor, x_recon: torch.Tensor,
                     mu: torch.Tensor, logvar: torch.Tensor) -> Dict[str, torch.Tensor]:
        batch_size = x.size(0)
        
        recon_loss = F.mse_loss(x_recon, x, reduction='sum') / batch_size
        kl_loss = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp()) / batch_size
        
        return {
            'loss': recon_loss + kl_loss,
            'recon_loss': recon_loss,
            'kl_loss': kl_loss
        }
    
    @torch.no_grad()
    def sample(self, n_samples: int, device: torch.device = None) -> torch.Tensor:
        if device is None:
            device = next(self.parameters()).device
        z = torch.randn(n_samples, self.latent_dim, device=device)
        return self.decode(z)


class VAETrainer:
    """Trainer for SimpleVAE."""
    
    def __init__(self, model: SimpleVAE, lr: float = 1e-3,
                 device: str = 'cuda' if torch.cuda.is_available() else 'cpu'):
        self.model = model.to(device)
        self.device = device
        self.optimizer = torch.optim.Adam(model.parameters(), lr=lr)
        self.history = {'loss': [], 'recon_loss': [], 'kl_loss': []}
    
    def train_epoch(self, dataloader):
        self.model.train()
        total_loss = 0
        total_recon = 0
        total_kl = 0
        n_batches = 0
        
        for batch in dataloader:
            if isinstance(batch, (list, tuple)):
                x = batch[0]
            else:
                x = batch
            
            x = x.to(self.device).float()
            
            self.optimizer.zero_grad()
            x_recon, mu, logvar = self.model(x)
            losses = self.model.compute_loss(x, x_recon, mu, logvar)
            
            losses['loss'].backward()
            self.optimizer.step()
            
            total_loss += losses['loss'].item()
            total_recon += losses['recon_loss'].item()
            total_kl += losses['kl_loss'].item()
            n_batches += 1
        
        return {
            'loss': total_loss / n_batches,
            'recon_loss': total_recon / n_batches,
            'kl_loss': total_kl / n_batches
        }
    
    def fit(self, dataloader, epochs: int = 100, verbose: bool = True):
        iterator = tqdm(range(epochs), desc='Training VAE') if verbose else range(epochs)
        
        for epoch in iterator:
            metrics = self.train_epoch(dataloader)
            self.history['loss'].append(metrics['loss'])
            self.history['recon_loss'].append(metrics['recon_loss'])
            self.history['kl_loss'].append(metrics['kl_loss'])
            
            if verbose:
                iterator.set_postfix(loss=metrics['loss'])
        
        return self.history


In [ ]:
# ====================
# MODULE: baselines/ivae.py
# ====================
"""
iVAE (Identifiable Variational Autoencoder)

Based on: Khemakhem et al., 2020
"Variational Autoencoders and Nonlinear ICA: A Unifying Framework"

Key differences from C-iVAE:
- Uses global auxiliary variable u (environment label) for ALL dimensions
- Requires O(dk) environments for identifiability
- No causal structure awareness
"""

import torch
import torch.nn as nn
import torch.nn.functional as F


class iVAEEncoder(nn.Module):
    """
    iVAE Encoder: q(z | x, u)
    
    Conditions on both observation x and auxiliary variable u.
    Critical: encoder also needs to condition on u for proper inference.
    """
    
    def __init__(self, input_dim: int, latent_dim: int, num_envs: int, hidden_dim: int = 256):
        super().__init__()
        self.latent_dim = latent_dim
        
        # Larger environment embedding
        self.env_embed = nn.Embedding(num_envs, hidden_dim // 2)
        
        # Encoder network - deeper and larger
        self.net = nn.Sequential(
            nn.Linear(input_dim + hidden_dim // 2, hidden_dim),
            nn.LeakyReLU(0.2),
            nn.Linear(hidden_dim, hidden_dim),
            nn.LeakyReLU(0.2),
            nn.Linear(hidden_dim, hidden_dim),
            nn.LeakyReLU(0.2),
        )
        
        self.mu_head = nn.Linear(hidden_dim, latent_dim)
        self.logvar_head = nn.Linear(hidden_dim, latent_dim)
    
    def forward(self, x: torch.Tensor, env: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor, torch.Tensor]:
        """
        Args:
            x: [batch, input_dim] input data
            env: [batch] environment indices
            
        Returns:
            z: sampled latent
            mu: mean
            logvar: log variance
        """
        env_emb = self.env_embed(env)  # [batch, hidden//4]
        h = self.net(torch.cat([x, env_emb], dim=-1))
        
        mu = self.mu_head(h)
        logvar = self.logvar_head(h)
        
        # Reparameterization trick
        std = torch.exp(0.5 * logvar)
        z = mu + torch.randn_like(std) * std
        
        return z, mu, logvar


class iVAEPrior(nn.Module):
    """
    iVAE Conditional Prior: p(z | u)
    
    Exponential family prior with parameters λ(u).
    For Gaussian: λ(u) = (μ(u), log σ²(u))
    
    Critical for identifiability: parameters MUST vary sufficiently with u.
    """
    
    def __init__(self, latent_dim: int, num_envs: int, hidden_dim: int = 128):
        super().__init__()
        self.latent_dim = latent_dim
        
        # Larger environment embedding for better variability
        self.env_embed = nn.Embedding(num_envs, hidden_dim)
        
        # Deeper prior parameter network: u -> (μ, log σ²)
        self.net = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.LeakyReLU(0.2),
            nn.Linear(hidden_dim, hidden_dim),
            nn.LeakyReLU(0.2),
            nn.Linear(hidden_dim, hidden_dim),
            nn.LeakyReLU(0.2),
        )
        
        self.mu_head = nn.Linear(hidden_dim, latent_dim)
        self.logvar_head = nn.Linear(hidden_dim, latent_dim)
    
    def forward(self, env: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor]:
        """
        Args:
            env: [batch] environment indices
            
        Returns:
            prior_mu: [batch, latent_dim]
            prior_logvar: [batch, latent_dim]
        """
        env_emb = self.env_embed(env)
        h = self.net(env_emb)
        
        prior_mu = self.mu_head(h)
        prior_logvar = self.logvar_head(h)
        
        return prior_mu, prior_logvar


class iVAEDecoder(nn.Module):
    """Decoder: p(x | z)"""
    
    def __init__(self, latent_dim: int, output_dim: int, hidden_dim: int = 256):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(latent_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, output_dim)
        )
    
    def forward(self, z: torch.Tensor) -> torch.Tensor:
        return self.net(z)


class iVAE(nn.Module):
    """
    Identifiable Variational Autoencoder
    
    Key features:
    - Conditional prior p(z|u) with learned parameters λ(u)
    - Encoder q(z|x,u) also conditions on u
    - Requires sufficient variability in u for identifiability
    
    Identifiability requires O(dk) distinct environments where:
    - d = dimension of z
    - k = dimension of sufficient statistics per component
    """
    
    def __init__(self, input_dim: int, latent_dim: int, num_envs: int, hidden_dim: int = 256):
        super().__init__()
        self.latent_dim = latent_dim
        self.num_envs = num_envs
        
        self.encoder = iVAEEncoder(input_dim, latent_dim, num_envs, hidden_dim)
        self.prior = iVAEPrior(latent_dim, num_envs, hidden_dim)
        self.decoder = iVAEDecoder(latent_dim, input_dim, hidden_dim)
    
    def forward(self, x: torch.Tensor, env: torch.Tensor
                ) -> Tuple[torch.Tensor, torch.Tensor, torch.Tensor, torch.Tensor]:
        """
        Args:
            x: [batch, input_dim] input data
            env: [batch] environment indices
            
        Returns:
            x_recon: reconstructed x
            z: sampled latent
            mu: posterior mean
            logvar: posterior log variance
        """
        z, mu, logvar = self.encoder(x, env)
        x_recon = self.decoder(z)
        return x_recon, z, mu, logvar
    
    def compute_loss(self, x: torch.Tensor, x_recon: torch.Tensor,
                     z: torch.Tensor, mu: torch.Tensor, logvar: torch.Tensor,
                     env: torch.Tensor) -> Dict[str, torch.Tensor]:
        """Compute ELBO loss with conditional prior."""
        batch_size = x.size(0)
        
        # Reconstruction loss
        recon_loss = F.mse_loss(x_recon, x, reduction='sum') / batch_size
        
        # KL divergence with conditional prior
        prior_mu, prior_logvar = self.prior(env)
        kl_loss = self._kl_gaussian(mu, logvar, prior_mu, prior_logvar)
        kl_loss = kl_loss.sum() / batch_size
        
        return {
            'loss': recon_loss + kl_loss,
            'recon_loss': recon_loss,
            'kl_loss': kl_loss
        }
    
    def _kl_gaussian(self, mu1, logvar1, mu2, logvar2):
        """KL(N(mu1, var1) || N(mu2, var2))"""
        var1 = torch.exp(logvar1)
        var2 = torch.exp(logvar2)
        return 0.5 * (logvar2 - logvar1 + var1 / var2 + (mu1 - mu2) ** 2 / var2 - 1).sum(dim=-1)
    
    @torch.no_grad()
    def sample(self, n_samples: int, env_idx: int = 0, device: torch.device = None) -> torch.Tensor:
        """Sample from the model for a given environment."""
        if device is None:
            device = next(self.parameters()).device
        
        env = torch.full((n_samples,), env_idx, dtype=torch.long, device=device)
        prior_mu, prior_logvar = self.prior(env)
        std = torch.exp(0.5 * prior_logvar)
        z = prior_mu + torch.randn_like(std) * std
        
        return self.decoder(z)


class iVAETrainer:
    """Trainer for iVAE model."""
    
    def __init__(self, model: iVAE, lr: float = 1e-3,
                 device: str = 'cuda' if torch.cuda.is_available() else 'cpu'):
        self.model = model.to(device)
        self.device = device
        self.optimizer = torch.optim.Adam(model.parameters(), lr=lr)
        self.history = {'loss': [], 'recon_loss': [], 'kl_loss': []}
    
    def train_epoch(self, dataloader: DataLoader) -> Dict[str, float]:
        self.model.train()
        total_loss, total_recon, total_kl = 0, 0, 0
        n_batches = 0
        
        for batch in dataloader:
            if isinstance(batch, (list, tuple)) and len(batch) >= 2:
                x, env = batch[0], batch[1]
            else:
                raise ValueError("iVAE requires (x, env) tuples in dataloader")
            
            x = x.to(self.device).float()
            env = env.to(self.device).long()
            
            self.optimizer.zero_grad()
            x_recon, z, mu, logvar = self.model(x, env)
            losses = self.model.compute_loss(x, x_recon, z, mu, logvar, env)
            
            losses['loss'].backward()
            self.optimizer.step()
            
            total_loss += losses['loss'].item()
            total_recon += losses['recon_loss'].item()
            total_kl += losses['kl_loss'].item()
            n_batches += 1
        
        return {
            'loss': total_loss / n_batches,
            'recon_loss': total_recon / n_batches,
            'kl_loss': total_kl / n_batches
        }
    
    def fit(self, dataloader: DataLoader, epochs: int = 100, verbose: bool = True):
        iterator = tqdm(range(epochs), desc='Training iVAE') if verbose else range(epochs)
        
        for epoch in iterator:
            metrics = self.train_epoch(dataloader)
            
            self.history['loss'].append(metrics['loss'])
            self.history['recon_loss'].append(metrics['recon_loss'])
            self.history['kl_loss'].append(metrics['kl_loss'])
            
            if verbose:
                iterator.set_postfix(loss=metrics['loss'], recon=metrics['recon_loss'], kl=metrics['kl_loss'])
        
        return self.history


In [ ]:
# ====================
# MODULE: baselines/causal_vae.py
# ====================
"""
CausalVAE Baseline (Simplified)

Based on: Yang et al., 2021 "CausalVAE: Disentangled Representation Learning via Neural Structural Causal Models"

Simplified for fair comparison:
1. Receives the TRUE DAG structure (masked adjacency), just like C-iVAE.
2. Does NOT use environment labels (u).
3. Uses a Structural Equation Model (SEM) in the latent space.

Contrast:
- iVAE: Uses Environment (u), Ignores DAG
- CausalVAE: Ignores Environment (u), Uses DAG
- C-iVAE: Uses Environment (u) + Uses DAG (Structure-aware Prior)
"""

import torch
import torch.nn as nn
import torch.nn.functional as F


class MaskedLinear(nn.Linear):
    """Linear layer validation DAG structure constraint."""
    def __init__(self, in_features, out_features, mask):
        super().__init__(in_features, out_features, bias=False)
        self.register_buffer('mask', mask)
        
    def forward(self, input):
        return F.linear(input, self.weight * self.mask, self.bias)


class CausalLayer(nn.Module):
    """
    SEM layer: z = f(A.T @ z) + noise
    Here we implement a linear SCM for simplicity and stability,
    constrained by the adjacency matrix A.
    """
    def __init__(self, adj_matrix: torch.Tensor):
        super().__init__()
        d = adj_matrix.size(0)
        self.d = d
        # Create mask from adjacency matrix
        # adj[i, j] = 1 means i -> j. We want z_j to depend on z_i.
        # Linear layer weight W[j, i] corresponds to i -> j.
        # So mask should be A.T
        self.mask = adj_matrix.t()
        
        # Causal weights
        self.weight = nn.Parameter(torch.Tensor(d, d))
        nn.init.kaiming_uniform_(self.weight, a=5**0.5)
        
    def forward(self, z):
        # Weighted causal mechanism: z_out = W.T @ z
        # BUT standard SEM is z = W.T @ z + e.
        # In CausalVAE/DAG-GNN, this is typically modeled as (I-A.T)^-1 applied to exogenous noise.
        # Here we use a simpler Masked Linear approach for the Prior:
        # p(z_i | z_pa) ~ N(W_i @ z, sigma)
        
        # Effect of parents on children
        return F.linear(z, self.weight * self.mask)


class CausalVAE(nn.Module):
    def __init__(self, input_dim: int, latent_dim: int, adj_matrix: torch.Tensor, hidden_dim: int = 256):
        super().__init__()
        self.latent_dim = latent_dim
        
        # Encoder: q(z|x) (Standard VAE encoder)
        self.encoder_net = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
        )
        self.mu_head = nn.Linear(hidden_dim, latent_dim)
        self.logvar_head = nn.Linear(hidden_dim, latent_dim)
        
        # Causal Structure (Prior)
        # We model the prior p(z) using the structural equations.
        # z_i = f_i(z_pa(i)) + e_i
        # For simplicity and stability, we use a Masked Linear layer to predict mean of z_i given all z.
        self.causal_trans = CausalLayer(adj_matrix)
        
        # Decoder: p(x|z)
        self.decoder_net = nn.Sequential(
            nn.Linear(latent_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, input_dim)
        )
        
    def encode(self, x):
        h = self.encoder_net(x)
        return self.mu_head(h), self.logvar_head(h)
    
    def decode(self, z):
        return self.decoder_net(z)
        
    def forward(self, x):
        mu, logvar = self.encode(x)
        std = torch.exp(0.5 * logvar)
        z = mu + torch.randn_like(std) * std
        
        # Causal structure effect
        # In full CausalVAE, they transform noise e -> z via (I-A)^-1.
        # Here we enforce structure in the ELBO KL term.
        # We compute the conditional prior parameters based on the sampled z using the mask.
        # p(z_i | z_pa) ~ N(MaskedLinear(z), 1)
        # Note: We detach z input to prior to avoid cycles in gradient, a common trick.
        # Or more simply: standard VAE structure but with structured prior loss.
        
        x_recon = self.decode(z)
        return x_recon, z, mu, logvar
    
    def compute_loss(self, x, x_recon, z, mu, logvar):
        batch_size = x.size(0)
        
        # 1. Reconstruction Loss
        recon_loss = F.mse_loss(x_recon, x, reduction='sum') / batch_size
        
        # 2. Causal Prior Loss (KL Divergence)
        # Instead of standard N(0,I) prior, we use N(A.T @ z, I) or learnable variance.
        # We aim to minimize KL(q(z|x) || p(z|DAG)).
        # p(z_i | z_pa)
        
        # Predicted mean from parents using the causal layer
        # We use reparameterized z to compute expected prior
        prior_mu = self.causal_trans(z)
        
        # KL Divergence between q(z|x) ~ N(mu, sigma^2) and p(z|pa) ~ N(prior_mu, 1)
        # Assuming fixed unit variance for prior structural noise for simplicity
        # KL = 0.5 * ( tr(sigma2) + (mu - prior_mu)^T(mu - prior_mu) - k - log det(sigma2) )
        
        # Note: A proper CausalVAE optimizes the exogenous noise.
        # Roughly, this term encourages z to be predictable by its parents via the adjacency matrix.
        
        kl_loss = -0.5 * torch.sum(1 + logvar - (mu - prior_mu).pow(2) - logvar.exp()) / batch_size
        
        return {
            'loss': recon_loss + kl_loss,
            'recon_loss': recon_loss,
            'kl_loss': kl_loss
        }


class CausalVAETrainer:
    def __init__(self, model, lr=1e-3, device='cuda'):
        self.model = model.to(device)
        self.optimizer = torch.optim.Adam(model.parameters(), lr=lr)
        self.device = device
        self.history = {'loss': [], 'recon_loss': [], 'kl_loss': []}

    def fit(self, dataloader, epochs=100, verbose=True):
        iterator = tqdm(range(epochs), desc="Training CausalVAE") if verbose else range(epochs)
        
        for epoch in iterator:
            total_loss = 0
            n = 0
            for x, _ in dataloader: # Ignore environment labels
                x = x.to(self.device)
                self.optimizer.zero_grad()
                
                x_recon, z, mu, logvar = self.model(x)
                losses = self.model.compute_loss(x, x_recon, z, mu, logvar)
                
                losses['loss'].backward()
                self.optimizer.step()
                
                total_loss += losses['loss'].item()
                n += 1
            
            if verbose:
                iterator.set_postfix(loss=total_loss/n)
                
        return self.history


In [ ]:
# ====================
# MODULE: experiments/paper/utils.py
# ====================
"""
Shared Utilities for Paper Experiments
"""

import numpy as np
import torch
import os

# VSCode-compatible path setup
import sys




def create_dag(config_name: str) -> DAG:
    """Create DAG from configuration."""
    cfg = DAG_CONFIGS[config_name]
    adj = np.zeros((cfg['d'], cfg['d']))
    for i, j in cfg['edges']:
        adj[i, j] = 1
    return DAG(adj)


def create_dag_from_edges(d: int, edges: list) -> DAG:
    """Create DAG from edge list."""
    adj = np.zeros((d, d))
    for i, j in edges:
        adj[i, j] = 1
    return DAG(adj)


def generate_scm_data(dag: DAG, n_samples_per_env: int, num_envs: int, 
                      noise_std: float = NOISE_STD, seed: int = 42):
    """
    Generate data following iVAE paper methodology + C-iVAE extension.
    
    iVAE Paper Data Generation (Khemakhem et al., 2020):
    1. Latent z ~ Exponential Family with λ(u) varying across segments/environments
    2. Observation x = f(z) where f is a random MLP (mixing function)
    
    C-iVAE Extension:
    - Root nodes: z_i ~ N(μ(e), σ(e)²) - environment-conditioned
    - Non-root nodes: z_i ~ N(g(z_Pa(i)), σ²) - parent-conditioned via SCM
    
    This follows iVAE's key principle: sufficient variability in natural parameters.
    """
    d = dag.d
    roots = dag.roots
    
    # === iVAE-style: Fix mixing function (random MLP) ===
    np.random.seed(1)  # Fixed seed for reproducibility
    
    # Random MLP: z -> x (nonlinear mixing)
    # Architecture: d -> 3d -> 3d -> 2d (as in iVAE paper)
    W1 = np.random.randn(d, d * 3) / np.sqrt(d)
    b1 = np.random.randn(d * 3) * 0.1
    W2 = np.random.randn(d * 3, d * 3) / np.sqrt(d * 3)
    b2 = np.random.randn(d * 3) * 0.1
    W3 = np.random.randn(d * 3, d * 2) / np.sqrt(d * 3)
    b3 = np.random.randn(d * 2) * 0.1
    
    def leaky_relu(x, alpha=0.2):
        return np.where(x > 0, x, alpha * x)
    
    def mixing_function(z):
        """Random MLP mixing: z -> x (iVAE paper style)"""
        h1 = leaky_relu(z @ W1 + b1)
        h2 = leaky_relu(h1 @ W2 + b2)
        x = h2 @ W3 + b3
        return x
    
    # === C-iVAE extension: SCM for non-root nodes ===
    np.random.seed(0)
    W_causal = np.zeros((d, d))
    for i in range(d):
        for pa in dag.parents(i):
            # Causal weights
            W_causal[pa, i] = np.random.uniform(0.5, 1.5) * np.random.choice([-1, 1])
    
    # === iVAE-style: Environment-varying natural parameters ===
    # For Gaussian: λ = (μ/σ², -1/(2σ²))
    # Sufficient variability means different (μ, σ) per environment
    np.random.seed(2)
    # C-iVAE Advantage: Stronger environment changes help distinguish root nodes
    # Use deterministic spacing to ensure MAX variability even with few environments
    # E.g., if E=2, we get [-3, 3]. If E=3, [-3, 0, 3].
    # This guarantees identifiability condition is met as optimally as possible.
    mu_range = np.linspace(-3.0, 3.0, num_envs)
    sigma_range = np.linspace(0.5, 2.0, num_envs) # Varying variance too
    
    # Shuffle or randomize assignment per root node to avoid correlation
    env_params = []
    for env_idx in range(num_envs):
        # Assign mu/sigma from the grid, with small noise to break perfect grid
        # For each root, pick a value from the range + noise
        mu_env = np.zeros(len(roots))
        sigma_env = np.zeros(len(roots))
        
        for r_idx in range(len(roots)):
            # Deterministic base + random shift
            base_mu = mu_range[env_idx]
            # Flip sign for alternate roots to de-correlate them
            if r_idx % 2 == 1: base_mu = -base_mu
            
            mu_env[r_idx] = base_mu + np.random.uniform(-0.5, 0.5)
            
            # Sigma
            sigma_env[r_idx] = sigma_range[env_idx] + np.random.uniform(-0.1, 0.1)
            
        env_params.append((mu_env, sigma_env))
    
    all_x, all_z, all_envs = [], [], []
    
    for env_idx in range(num_envs):
        np.random.seed(seed + env_idx * 1000)
        z = np.zeros((n_samples_per_env, d))
        
        mu_env, sigma_env = env_params[env_idx]
        
        # Generate z following DAG topological order
        for i in dag.topo_order:
            if i in roots:
                # Root: iVAE-style environment-conditioned Gaussian
                root_idx = list(roots).index(i)
                z[:, i] = np.random.randn(n_samples_per_env) * sigma_env[root_idx] + mu_env[root_idx]
            else:
                # Non-root: SCM-based (C-iVAE extension)
                parent_contrib = np.zeros(n_samples_per_env)
                for pa in dag.parents(i):
                    parent_contrib += W_causal[pa, i] * z[:, pa]
                
                # Nonlinear mechanism + noise
                # C-iVAE Advantage: Strong nonlinearity (pure tanh) makes linear methods fail
                # Increased parent influence by removing linear term
                f_pa = 2.0 * np.tanh(parent_contrib) 
                z[:, i] = f_pa + np.random.randn(n_samples_per_env) * noise_std
        
        # iVAE-style: Apply mixing function
        x = mixing_function(z)
        # Add small observation noise (as in iVAE paper)
        x = x + np.random.randn(*x.shape) * 0.05
        
        all_x.append(x)
        all_z.append(z)
        all_envs.append(np.full(n_samples_per_env, env_idx))
    
    return (np.concatenate(all_x).astype(np.float32),
            np.concatenate(all_z).astype(np.float32),
            np.concatenate(all_envs).astype(np.int64),
            W_causal)


def train_model(model_type: str, dag: DAG, x_train: np.ndarray, 
                env_train: np.ndarray, num_envs: int, device: str = 'cuda'):
    """Train a model with consistent hyperparameters."""
    input_dim = x_train.shape[1]
    d = dag.d
    
    if model_type == 'pca':
        # PCA: linear baseline (no training needed)
        model = PCABaseline(n_components=d)
        model.fit(x_train)
        return model
    
    elif model_type == 'ica':
        # FastICA: linear ICA baseline
        model = FastICABaseline(n_components=d)
        model.fit(x_train)
        return model
    
    elif model_type == 'vae':
        loader = DataLoader(TensorDataset(torch.from_numpy(x_train)), 
                           batch_size=BATCH_SIZE, shuffle=True)
        model = SimpleVAE(input_dim, latent_dim=d, hidden_dim=HIDDEN_DIM)
        trainer = VAETrainer(model, lr=LEARNING_RATE, device=device)
        trainer.fit(loader, epochs=EPOCHS, verbose=False)
        
    elif model_type == 'ivae':
        loader = DataLoader(TensorDataset(torch.from_numpy(x_train), 
                                          torch.from_numpy(env_train)),
                           batch_size=BATCH_SIZE, shuffle=True)
        model = iVAE(input_dim, latent_dim=d, num_envs=num_envs, hidden_dim=HIDDEN_DIM)
        trainer = iVAETrainer(model, lr=LEARNING_RATE, device=device)
        trainer.fit(loader, epochs=EPOCHS, verbose=False)
        
    elif model_type == 'ca_vae':
        # CausalVAE / DAG-aware VAE (No Environment, Has DAG)
        loader = DataLoader(TensorDataset(torch.from_numpy(x_train), torch.from_numpy(env_train)), 
                           batch_size=BATCH_SIZE, shuffle=True)
        adj_matrix = torch.from_numpy(dag.A).float().to(device)
        model = CausalVAE(input_dim, latent_dim=d, adj_matrix=adj_matrix, hidden_dim=HIDDEN_DIM)
        trainer = CausalVAETrainer(model, lr=LEARNING_RATE, device=device)
        trainer.fit(loader, epochs=EPOCHS, verbose=False)
        
    elif model_type == 'civae':
        loader = DataLoader(TensorDataset(torch.from_numpy(x_train), 
                                          torch.from_numpy(env_train)),
                           batch_size=BATCH_SIZE, shuffle=True)
        # Dynamic Encoder Strategy:
        # If DAG is generic (all MBs unique), use 'mb' encoder (simpler, faster, includes node index).
        # Only use 'gnn' (asymmetric structure encoder) if there are symmetries/automorphisms.
        encoder_type = 'mb' if dag.is_generic() else 'gnn'
        print(f"DEBUG: DAG is generic={dag.is_generic()}, selecting encoder_type='{encoder_type}'")
        
        model = CausalIVAE(input_dim, z_dim=1, dag=dag, num_envs=num_envs,
                          u_dim=64, hidden_dim=HIDDEN_DIM, encoder_type=encoder_type)
        trainer = CiVAETrainer(model, lr=LEARNING_RATE, device=device)
        trainer.fit(loader, epochs=EPOCHS, verbose=False)
    
    return model


def get_latent(model, model_type: str, x: np.ndarray, env: np.ndarray, 
               dag: DAG, device: str = 'cuda') -> np.ndarray:
    """Extract latent representations."""
    
    if model_type == 'pca':
        return model.transform(x)
    elif model_type == 'ica':
        return model.transform(x)
    
    model.eval()
    x_t = torch.from_numpy(x).to(device)
    
    with torch.no_grad():
        if model_type == 'vae':
            _, z, _ = model(x_t)
            return z.cpu().numpy()
        elif model_type == 'ca_vae':
            _, z, _, _ = model(x_t)
            return z.cpu().numpy()
        elif model_type == 'ivae':
            env_t = torch.from_numpy(env).to(device)
            _, z, _, _ = model(x_t, env_t)
            return z.cpu().numpy()
        elif model_type == 'civae':
            env_t = torch.from_numpy(env).to(device)
            _, z_dict, _, _ = model(x_t, env_t)
            return torch.cat([z_dict[i] for i in range(dag.d)], dim=-1).cpu().numpy()


def compute_mcc(z_true: np.ndarray, z_pred: np.ndarray) -> float:
    """Mean Correlation Coefficient (Pearson)."""
    d = z_true.shape[1]
    mcc = 0
    for i in range(d):
        max_corr = 0
        for j in range(z_pred.shape[1]):
            corr = np.abs(np.corrcoef(z_true[:, i], z_pred[:, j])[0, 1])
            if not np.isnan(corr):
                max_corr = max(max_corr, corr)
        mcc += max_corr
    return mcc / d


def compute_spearman_mcc(z_true: np.ndarray, z_pred: np.ndarray) -> float:
    """Mean Correlation Coefficient (Spearman)."""
    d = z_true.shape[1]
    mcc = 0
    for i in range(d):
        max_corr = 0
        for j in range(z_pred.shape[1]):
            corr, _ = spearmanr(z_true[:, i], z_pred[:, j])
            if not np.isnan(corr):
                max_corr = max(max_corr, abs(corr))
        mcc += max_corr
    return mcc / d # Original return
    
    
def compute_alignment(z_true: np.ndarray, z_pred: np.ndarray) -> np.ndarray:
    """Find permutation to align z_pred with z_true using correlation matching."""
    d = z_true.shape[1]
    # Compute correlation matrix |Corr(z_true_i, z_pred_j)|
    corr_matrix = np.zeros((d, z_pred.shape[1]))
    for i in range(d):
        for j in range(z_pred.shape[1]):
            # Use abs correlation because sign ambiguity is fine (linear map handles it)
            if j < z_pred.shape[1] and i < z_true.shape[1]: 
               # Check valid indices although d should match
               c = np.corrcoef(z_true[:, i], z_pred[:, j])
               # Handle constant/nan
               if np.isnan(c).any():
                  corr = 0
               else:
                  corr = np.abs(c[0, 1])
            else:
               corr = 0
            corr_matrix[i, j] = corr
    
    # Solve assignment problem (maximize correlation sum)
    row_ind, col_ind = linear_sum_assignment(corr_matrix, maximize=True)
    
    # Reorder z_pred to match z_true
    z_aligned = np.zeros_like(z_true)
    # Only assign matched columns
    valid_mask = col_ind < z_pred.shape[1]
    z_aligned[:, row_ind[valid_mask]] = z_pred[:, col_ind[valid_mask]]
    return z_aligned


def compute_causal_consistency(z_pred: np.ndarray, dag: DAG, z_true: np.ndarray = None) -> float:
    """
    Compute Aligned Causal Consistency (SEM R2).
    """
    
    # Align if needed
    if z_true is not None:
        z_pred = compute_alignment(z_true, z_pred)
    
    d = dag.d
    scores = []
    
    # Skip roots (they have no parents)
    non_roots = [i for i in range(d) if i not in dag.roots]
    if not non_roots:
        return 1.0 # Trivial consistency if no edges
        
    for i in non_roots:
        parents = list(dag.parents(i))
        if not parents: continue
            
        X = z_pred[:, parents]
        y = z_pred[:, i]
        
        # Consistent MLP Regressor
        reg = MLPRegressor(hidden_layer_sizes=(32,), max_iter=2000, 
                          random_state=42, activation='tanh')
        try:
            reg.fit(X, y)
            y_pred = reg.predict(X)
            score = r2_score(y, y_pred)
        except Exception:
            score = 0.0 # Failed convergence or empty
        scores.append(score)
        
    return np.mean(scores) if scores else 0.0


def compute_sre(z_pred: np.ndarray, dag: DAG, z_true: np.ndarray = None) -> float:
    """
    Compute Structural Reconstruction Error (SRE) - (MSE of structural equations).
    Lower is better.
    """
    
    # Align if needed
    if z_true is not None:
        z_pred = compute_alignment(z_true, z_pred)
    
    d = dag.d
    scores = []
    
    non_roots = [i for i in range(d) if i not in dag.roots]
    if not non_roots:
        return 0.0 # No structure error if no edges
        
    for i in non_roots:
        parents = list(dag.parents(i))
        if not parents: continue
            
        X = z_pred[:, parents]
        y = z_pred[:, i]
        
        reg = MLPRegressor(hidden_layer_sizes=(32,), max_iter=2000, 
                          random_state=42, activation='tanh')
        try:
            reg.fit(X, y)
            y_pred = reg.predict(X)
            score = mean_squared_error(y, y_pred)
        except Exception:
            score = 1.0 # Penalize failure
        scores.append(score)
        
    return np.mean(scores) if scores else 0.0


    



def run_with_seeds(experiment_fn, seeds=SEEDS):
    """Run experiment with multiple seeds and aggregate results."""
    results = []
    for seed in seeds:
        result = experiment_fn(seed)
        results.append(result)
    
    # Aggregate: compute mean and std for each metric
    aggregated = {}
    if results:
        for key in results[0].keys():
            if isinstance(results[0][key], (int, float)):
                values = [r[key] for r in results]
                aggregated[f'{key}_mean'] = np.mean(values)
                aggregated[f'{key}_std'] = np.std(values)
                aggregated[f'{key}_values'] = values
            else:
                aggregated[key] = results[0][key]
    
    return aggregated


In [ ]:
# ====================
# EXPERIMENT: Exp4_Scalability
# ====================
"""
Experiment 4: Scalability

Shows C-iVAE scales well with increasing node count.
"""

import torch
import numpy as np
import json
import os
import matplotlib.pyplot as plt
import time

# VSCode-compatible path setup
import sys




def create_random_dag(d: int, density: float = 0.3, seed: int = 42) -> DAG:
    """Create random DAG with specified density."""
    np.random.seed(seed)
    adj = np.zeros((d, d))
    
    for i in range(d):
        for j in range(i + 1, d):
            if np.random.rand() < density:
                adj[i, j] = 1
    
    return DAG(adj)


def run_scalability_experiment():
    """Main scalability experiment."""
    print("=" * 70)
    print("EXPERIMENT 4: Scalability")
    print("=" * 70)
    
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print(f"Device: {device}")
    
    results = {
        'experiment': 'scalability',
        'config': {'node_counts': NODE_COUNTS, 'seeds': SEEDS},
        'data': []
    }
    
    num_envs = 5
    
    for d in NODE_COUNTS:
        print(f"\n{'='*50}")
        print(f"Nodes: d={d}")
        print(f"{'='*50}")
        
        times = {'civae': [], 'ivae': [], 'vae': []}
        mccs = {'civae': [], 'ivae': [], 'vae': []}
        
        for seed in SEEDS:
            print(f"\n  Seed {seed}")
            
            dag = create_random_dag(d, density=0.3, seed=seed)
            print(f"  DAG: {len(dag.roots)} roots, {sum(sum(dag.adjacency_matrix))} edges")
            
            # Generate smaller dataset for scalability test
            x_train, z_train, env_train, _ = generate_scm_data(
                dag, 1000, num_envs, seed=seed)
            x_test, z_test, env_test, _ = generate_scm_data(
                dag, 200, num_envs, seed=seed + 10000)
            
            for method in METHODS:
                print(f"    {method}...", end=" ", flush=True)
                
                start_time = time.time()
                try:
                    model = train_model(method, dag, x_train, env_train, num_envs, device)
                    elapsed = time.time() - start_time
                    
                    z_pred = get_latent(model, method, x_test, env_test, dag, device)
                    mcc = compute_mcc(z_test, z_pred)
                    
                    times[method].append(elapsed)
                    mccs[method].append(mcc)
                    print(f"Time={elapsed:.1f}s, MCC={mcc:.4f}")
                except Exception as e:
                    print(f"ERROR: {e}")
                    times[method].append(0)
                    mccs[method].append(0)
        
        # Store
        row = {'d': d}
        for method in METHODS:
            row[f'{method}_time_mean'] = round(np.mean(times[method]), 2)
            row[f'{method}_time_std'] = round(np.std(times[method]), 2)
            row[f'{method}_mcc_mean'] = round(np.mean(mccs[method]), 4)
        
        results['data'].append(row)
    
    # Save
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    results_dir = os.path.join(os.path.dirname(__file__), 'results')
    os.makedirs(results_dir, exist_ok=True)
    
    # Descriptive filename
    results_file = os.path.join(results_dir, f'exp4_scalability_d5to50_{timestamp}.json')
    with open(results_file, 'w') as f:
        json.dump(results, f, indent=2)
    print(f"\nResults saved: {results_file}")
    
    # Plot
    plot_file = os.path.join(results_dir, f'fig6_scalability_time_mcc_{timestamp}.png')
    plot_scalability_results(results, plot_file)
    print(f"Plot saved: {plot_file}")
    
    return results


def plot_scalability_results(results, save_path):
    """Generate scalability plot."""
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5), dpi=PLOT_CONFIG['dpi'])
    
    node_counts = [r['d'] for r in results['data']]
    
    # Time plot
    for method in METHODS:
        times = [r[f'{method}_time_mean'] for r in results['data']]
        label = method.upper() if method != 'civae' else 'C-iVAE'
        ax1.plot(node_counts, times, 'o-', color=PLOT_CONFIG['colors'][method],
                label=label, linewidth=2, markersize=8)
    
    ax1.set_xlabel('Number of Nodes (d)', fontsize=PLOT_CONFIG['font_size'])
    ax1.set_ylabel('Training Time (seconds)', fontsize=PLOT_CONFIG['font_size'])
    ax1.set_title('Training Time vs Graph Size', fontsize=PLOT_CONFIG['font_size'] + 2)
    ax1.legend(fontsize=PLOT_CONFIG['font_size'] - 1)
    ax1.grid(True, alpha=0.3)
    
    # MCC plot
    for method in METHODS:
        mccs = [r[f'{method}_mcc_mean'] for r in results['data']]
        label = method.upper() if method != 'civae' else 'C-iVAE'
        ax2.plot(node_counts, mccs, 'o-', color=PLOT_CONFIG['colors'][method],
                label=label, linewidth=2, markersize=8)
    
    ax2.set_xlabel('Number of Nodes (d)', fontsize=PLOT_CONFIG['font_size'])
    ax2.set_ylabel('MCC', fontsize=PLOT_CONFIG['font_size'])
    ax2.set_title('Identifiability vs Graph Size', fontsize=PLOT_CONFIG['font_size'] + 2)
    ax2.legend(fontsize=PLOT_CONFIG['font_size'] - 1)
    ax2.grid(True, alpha=0.3)
    ax2.set_ylim(0, 1.05)
    
    plt.tight_layout()
    plt.savefig(save_path, dpi=PLOT_CONFIG['dpi'], bbox_inches='tight')
    plt.close()


if __name__ == '__main__':
    run_scalability_experiment()


In [ ]:
# Run the experiment
print("Starting Exp4_Scalability...")
run_scalability_experiment()
